# United outdoors datawarehouse

## Imports

In [485]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, BigInteger, Integer, String, LargeBinary, VARCHAR, NVARCHAR, DECIMAL, CHAR, \
    DATE
from sqlalchemy.dialects.mssql import BIT, XML, MONEY, TIME
from sqlalchemy.exc import OperationalError
from urllib import parse
import re
import time

## Starting timer

In [486]:
start_time = time.time()

## Database connection details

In [487]:
DB = {
    'servername' : '(local)\\SQLEXPRESS',
    'united_outdoors_database' : 'UnitedOutdoors',
    'northwind_database' : 'Northwind',
    'aenc_database' : 'Aenc',
    'adventureworks_database' : 'AdventureWorks2019',
    'master' : 'master'
}

In [488]:
def create_connection(servername, database):
    params = parse.quote_plus(f'DRIVER={{SQL Server}};SERVER={servername};DATABASE={database};Trusted_Connection=yes')
    engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}', use_setinputsizes=False, connect_args={'options': '-c search_path=dbo'}, fast_executemany=True) # setinputsizes needs to be turned off for sql server, idk why but gives errors otherwise
    try:
        establish_conn = engine.connect()
        print(f'Connection to {database} database successful')
        return establish_conn, engine
    except OperationalError as e:
        print(f'Error: {e}')
        return None, None

In [489]:
def split_and_execute_sql_script(script, execute_engine):
    # creating a connection
    connection = execute_engine.connect()
    
    # splitting the script into the database creation and the rest
    commands = re.split(r'GO\n', script)
    # removing all \bGO\b from the commands
    commands = [re.sub(r'\bGO\b', '', command) for command in commands]
    
    # Execute the commands
    for command in commands:
        command = command.strip()
        # Skip if the command is empty or 'GO'
        if not command or command.upper() == 'GO':
            continue
        try:
            connection.connection.execute(command)
            connection.connection.commit()
            #print(f'Command executed: {command}')
        except OperationalError as e:
            print(f'Error: {e} at command: {command}')

In [490]:
def bulk_insert(df, dtypes, table_name, engine):
    # calculating chunk size
    chunk_size = (2000 // len(df.columns))  # 2100 is the maximum number of parameters in a query, -100 for safety

    print(f'Inserting data into table: {table_name} with chunk size: {chunk_size}')
    df.to_sql(name=table_name, schema='dbo', con=engine, if_exists='append', index=False, dtype=dtypes, method='multi', chunksize= chunk_size)

In [491]:
def prepare(dataframe, nk_sk_dict=None):
    # replacing the natural keys with the surrogate keys
    if nk_sk_dict:
        for column in nk_sk_dict:
            # Check for duplicate keys
            if len(nk_sk_dict[column]) != len(set(nk_sk_dict[column])):
                raise ValueError(f'Duplicate keys found in nk_sk_dict for column: {column}')
            else:
                print(f'Replacing natural keys with surrogate keys for column: {column}')
                for natural_key in nk_sk_dict[column]:
                    dataframe[column] = dataframe[column].replace(natural_key, nk_sk_dict[column][natural_key])
    
    # replace empty values with None
    dataframe = dataframe.where(pd.notnull(dataframe), None)
    dataframe = dataframe.replace({np.nan: None})
    
    # stripping all columns with string data
    dataframe = dataframe.map(lambda x: x.strip() if isinstance(x, str) else x)
    # replacing all empty strings with None
    dataframe = dataframe.replace(r'^\s*$', None, regex=True)

In [492]:
def get_nk_sk_dict(engine, natural_key_column, table_name):
   
    surrogate_key_column = table_name.capitalize() + "_sk"
    
    query = f'WITH RankedKeys AS (SELECT {natural_key_column}, {surrogate_key_column}, ROW_NUMBER() OVER(PARTITION BY {natural_key_column} ORDER BY {table_name}_datetime_added DESC) AS rn FROM {table_name}) SELECT {natural_key_column}, {surrogate_key_column} FROM RankedKeys WHERE rn = 1' # getting the latest surrogate key for each natural key, by datetime_added
    
    # turing the query into a dataframe
    nk_sk_df = pd.read_sql(query, engine).dropna()
    
    # converting the dataframe to a dictionary
    nk_sk_dict = nk_sk_df.set_index(natural_key_column)[surrogate_key_column].to_dict()
    
    return nk_sk_dict

In [493]:
def prepare_and_insert(dataframe, dtypes, table_name, insert_engine, nk_sk_dict=None):
    """
    Prepares the dataframe for insertion into the database and inserts it into the database.
        @param dataframe: The dataframe to be inserted into the database
        @param dtypes: The data types of the columns in the dataframe
        @param table_name: The name of the table in the database
        @param insert_engine: The engine to insert the data into
        @param nk_sk_dict: A 3d dictionary containing the natural keys and their corresponding surrogate keys, per column (so nk_sk_dict[column][natural_key] = surrogate_key)
    """
       
    prepare(dataframe, nk_sk_dict)
    
    # adding the data to the database
    bulk_insert(dataframe, dtypes, table_name, insert_engine)

In [494]:
def prepare_and_insert_return_sk(dataframe, dtypes, table_name, insert_engine, natural_key_column, nk_sk_dict=None):
    """
    Prepares the dataframe for insertion into the database and inserts it into the database.
        @param dataframe: The dataframe to be inserted into the database
        @param dtypes: The data types of the columns in the dataframe
        @param table_name: The name of the table in the database
        @param insert_engine: The engine to insert the data into
        @param natural_key_column: The name of the column containing the natural keys
        @param nk_sk_dict: A 3d dictionary containing the natural keys and their corresponding surrogate keys, per column (so nk_sk_dict[column][natural_key] = surrogate_key)
        @return: A dictionary containing the natural keys and their corresponding surrogate keys
    """
    
    prepare(dataframe, nk_sk_dict)
    
    # adding the data to the database
    bulk_insert(dataframe, dtypes, table_name, insert_engine)
    
    # getting the natural key and surrogate key dictionary
    current_nk_sk_dict = get_nk_sk_dict(insert_engine, natural_key_column, table_name)
    
    return current_nk_sk_dict

In [495]:
def prepare_and_update(table_name, update_engine, nk_sk_dict=None):   
    # creating a connection
    connection = update_engine.connect()

    # Start a transaction
    trans = connection.begin()

    try:
        # adding the data to the database
        print(f'Updating data in table: {table_name}')
        for column in nk_sk_dict:
            # Prepare a batch update statement
            update_stmts = []
            for natural_key in nk_sk_dict[column]:
                # TODO this would not work with updated data in datawarehouse, since this would replace the old and new data. Maybe check the datetime or something
                statement = f'UPDATE {table_name} SET {column} = {nk_sk_dict[column][natural_key]} WHERE {column} = {int(natural_key)}'
                update_stmts.append(statement)

                # If the number of parameters reaches the limit, execute the batch update and clear the list
                if len(update_stmts) * 2 >= 2000:  # Each update statement has 2 parameters, cause of the Column and Where clause
                    connection.connection.execute(";".join(update_stmts))
                    update_stmts = []

            # Execute the remaining update statements
            if update_stmts:
                connection.connection.execute(";".join(update_stmts))

        # Commit the transaction
        trans.commit()
    except:
        # Rollback the transaction in case of error
        trans.rollback()
        raise
    finally:
        # Close the connection
        connection.close()

In [496]:
def drop_modified_date_rowguid(dataframe):
    # dropping all columns with 'rowguid' in their name
    columns_to_drop_mr = dataframe.filter(like='rowguid').columns
    
    # dropping all columns with 'ModifiedDate' in their name
    columns_to_drop_mr = columns_to_drop_mr.append(dataframe.filter(like='ModifiedDate').columns)
    
    # dropping the columns
    dataframe.drop(columns=columns_to_drop_mr, inplace=True)

## Create the UnitedOutdoors datawarehouse

In [497]:
_, creation_engine = create_connection(DB["servername"], DB["master"])

# Open the SQL script file and read its contents
with open('sql/UnitedOutdoors_creation.sql', 'r') as file:
    sql_script = file.read()

split_and_execute_sql_script(sql_script, creation_engine)

creation_engine.dispose()

Connection to master database successful


## Connecting to the UnitedOutdoors datawarehouse

In [498]:
_ , united_outdoors_engine = create_connection(DB["servername"], DB["united_outdoors_database"])

Connection to UnitedOutdoors database successful


## Loading the data from the source databases

### Northwind database

#### Connection

In [499]:
northwind_conn, northwind_engine = create_connection(DB["servername"], DB["northwind_database"])

Connection to Northwind database successful


#### Loading data

In [500]:
# Load the data from the source database
northwind_categories = pd.read_sql('SELECT * FROM Categories', northwind_conn)
northwind_customer_customer_demo = pd.read_sql('SELECT * FROM CustomerCustomerDemo', northwind_conn)
northwind_customer_demographics = pd.read_sql('SELECT * FROM CustomerDemographics', northwind_conn)
northwind_customers = pd.read_sql('SELECT * FROM Customers', northwind_conn)
northwind_employees = pd.read_sql('SELECT *, CONVERT(VARCHAR(MAX), CONVERT(VARBINARY(MAX), Photo), 1) as EMPLOYEE_EMPLOYEES_PhotoHexString FROM Employees', northwind_conn) # image is deprecated, but still in northwind. Direct conversion from image to varchar is not possible, so we need to convert it to varbinary first
northwind_employee_territories = pd.read_sql('SELECT * FROM EmployeeTerritories', northwind_conn)
northwind_order_details = pd.read_sql('SELECT * FROM [Order Details]', northwind_conn)
northwind_orders = pd.read_sql('SELECT * FROM Orders', northwind_conn)
northwind_products = pd.read_sql('SELECT * FROM Products', northwind_conn)
northwind_region = pd.read_sql('SELECT * FROM Region', northwind_conn)
northwind_shippers = pd.read_sql('SELECT * FROM Shippers', northwind_conn)
northwind_suppliers = pd.read_sql('SELECT * FROM Suppliers', northwind_conn)
northwind_territories = pd.read_sql('SELECT * FROM Territories', northwind_conn)

northwind_conn.close()

### Aenc database

#### Connection

In [501]:
aenc_conn , aenc_engine = create_connection(DB["servername"], DB["aenc_database"])

Connection to Aenc database successful


#### Loading data

In [502]:
aenc_bonus              = pd.read_sql('SELECT * FROM Bonus', aenc_conn)
aenc_customer           = pd.read_sql('SELECT * FROM Customer', aenc_conn)
aenc_department         = pd.read_sql('SELECT * FROM Department', aenc_conn)
aenc_employee           = pd.read_sql('SELECT * FROM Employee', aenc_conn)
aenc_product            = pd.read_sql('SELECT * FROM Product', aenc_conn)
aenc_region             = pd.read_sql('SELECT * FROM Region', aenc_conn)
aenc_sales_order        = pd.read_sql('SELECT * FROM SalesOrder', aenc_conn)
aenc_sales_order_item   = pd.read_sql('SELECT * FROM SalesOrderItem', aenc_conn)
aenc_state              = pd.read_sql('SELECT * FROM State', aenc_conn)

aenc_conn.close()

### AdventureWorks database

#### Connection

In [503]:
adventureworks_conn, adventureworks_engine = create_connection(DB["servername"], DB["adventureworks_database"])

Connection to AdventureWorks2019 database successful


#### Loading data

In [504]:
adventureworks_humanresources_department = pd.read_sql('SELECT * FROM HumanResources.Department', adventureworks_conn)
adventureworks_humanresources_employee = pd.read_sql('SELECT * FROM HumanResources.Employee', adventureworks_conn)
adventureworks_humanresources_employeedepartmenthistory = pd.read_sql('SELECT * FROM HumanResources.EmployeeDepartmentHistory', adventureworks_conn)
adventureworks_humanresources_employeepayhistory = pd.read_sql('SELECT * FROM HumanResources.EmployeePayHistory', adventureworks_conn)
adventureworks_humanresources_jobcandidate = pd.read_sql('SELECT * FROM HumanResources.JobCandidate', adventureworks_conn)
adventureworks_humanresources_shift = pd.read_sql('SELECT * FROM HumanResources.Shift', adventureworks_conn)

In [505]:
adventureworks_person_address = pd.read_sql('SELECT AddressID, AddressLine1, AddressLine2, City, StateProvinceID, PostalCode, CAST(SpatialLocation AS VARCHAR(MAX)) AS SpatialLocation,rowguid, ModifiedDate   FROM Person.Address', adventureworks_conn)
adventureworks_person_address_type = pd.read_sql('SELECT * FROM Person.AddressType', adventureworks_conn)
adventureworks_person_businessentity = pd.read_sql('SELECT * FROM Person.BusinessEntity', adventureworks_conn)
adventureworks_person_businessentityaddress = pd.read_sql('SELECT * FROM Person.BusinessEntityAddress', adventureworks_conn)
adventureworks_person_businessentitycontact = pd.read_sql('SELECT * FROM Person.BusinessEntityContact', adventureworks_conn)
adventureworks_person_contacttype = pd.read_sql('SELECT * FROM Person.ContactType', adventureworks_conn)
adventureworks_person_countryregion = pd.read_sql('SELECT * FROM Person.CountryRegion', adventureworks_conn)
adventureworks_person_emailaddress = pd.read_sql('SELECT * FROM Person.EmailAddress', adventureworks_conn)
adventureworks_person_password = pd.read_sql('SELECT * FROM Person.Password', adventureworks_conn)
adventureworks_person_person = pd.read_sql('SELECT * FROM Person.Person', adventureworks_conn)
adventureworks_person_personphone = pd.read_sql('SELECT * FROM Person.PersonPhone', adventureworks_conn)
adventureworks_person_phonenumbertype = pd.read_sql('SELECT * FROM Person.PhoneNumberType', adventureworks_conn)
adventureworks_person_stateprovince = pd.read_sql('SELECT * FROM Person.StateProvince', adventureworks_conn)

In [506]:
adventureworks_production_bill_of_materials = pd.read_sql('SELECT * FROM Production.BillOfMaterials', adventureworks_conn)
adventureworks_production_culture = pd.read_sql('SELECT * FROM Production.Culture', adventureworks_conn)
adventureworks_production_document = pd.read_sql('SELECT * FROM Production.Document', adventureworks_conn)
adventureworks_production_illustration = pd.read_sql('SELECT * FROM Production.Illustration', adventureworks_conn)
adventureworks_production_location = pd.read_sql('SELECT * FROM Production.Location', adventureworks_conn)
adventureworks_production_product = pd.read_sql('SELECT * FROM Production.Product', adventureworks_conn)
adventureworks_production_productcategory = pd.read_sql('SELECT * FROM Production.ProductCategory', adventureworks_conn)
adventureworks_production_productcosthistory = pd.read_sql('SELECT * FROM Production.ProductCostHistory', adventureworks_conn)
adventureworks_production_productdescription = pd.read_sql('SELECT * FROM Production.ProductDescription', adventureworks_conn)
adventureworks_production_productdocument = pd.read_sql('SELECT * , CAST(DocumentNode AS VARCHAR(MAX)) AS DocumentNodeString  FROM Production.ProductDocument', adventureworks_conn)
adventureworks_production_productinventory = pd.read_sql('SELECT * FROM Production.ProductInventory', adventureworks_conn)
adventureworks_production_productlistpricehistory = pd.read_sql('SELECT * FROM Production.ProductListPriceHistory', adventureworks_conn)
adventureworks_production_productmodel = pd.read_sql('SELECT * FROM Production.ProductModel', adventureworks_conn)
adventureworks_production_productmodelillustration = pd.read_sql('SELECT * FROM Production.ProductModelIllustration', adventureworks_conn)
adventureworks_production_productmodelproductdescriptionculture = pd.read_sql('SELECT * FROM Production.ProductModelProductDescriptionCulture', adventureworks_conn)
adventureworks_production_productphoto = pd.read_sql('SELECT ProductPhotoID, CONVERT(VARCHAR(MAX),ThumbNailPhoto, 1) as ThumbNailPhotoHexString, ThumbNailPhotoFileName, CONVERT(VARCHAR(MAX), LargePhoto, 1) as LargePhotoHexString, LargePhotoFileName, ModifiedDate FROM Production.ProductPhoto', adventureworks_conn)
adventureworks_production_productproductphoto = pd.read_sql('SELECT * FROM Production.ProductProductPhoto', adventureworks_conn)
adventureworks_production_productreview = pd.read_sql('SELECT * FROM Production.ProductReview', adventureworks_conn)
adventureworks_production_productsubcategory = pd.read_sql('SELECT * FROM Production.ProductSubcategory', adventureworks_conn)
adventureworks_production_scrapreason = pd.read_sql('SELECT * FROM Production.ScrapReason', adventureworks_conn)
adventureworks_production_transactionhistory = pd.read_sql('SELECT * FROM Production.TransactionHistory', adventureworks_conn)
adventureworks_production_transactionhistoryarchive = pd.read_sql('SELECT * FROM Production.TransactionHistoryArchive', adventureworks_conn)
adventureworks_production_unitmeasure = pd.read_sql('SELECT * FROM Production.UnitMeasure', adventureworks_conn)
adventureworks_production_workorder = pd.read_sql('SELECT * FROM Production.WorkOrder', adventureworks_conn)
adventureworks_production_workorderrouting = pd.read_sql('SELECT * FROM Production.WorkOrderRouting', adventureworks_conn)

In [507]:
adventureworks_purchasing_productvendor = pd.read_sql('SELECT * FROM Purchasing.ProductVendor', adventureworks_conn)
adventureworks_purchasing_purchaseorderdetail = pd.read_sql('SELECT * FROM Purchasing.PurchaseOrderDetail', adventureworks_conn)
adventureworks_purchasing_purchaseorderheader = pd.read_sql('SELECT * FROM Purchasing.PurchaseOrderHeader', adventureworks_conn)
adventureworks_purchasing_shipmethod = pd.read_sql('SELECT * FROM Purchasing.ShipMethod', adventureworks_conn)
adventureworks_purchasing_vendor = pd.read_sql('SELECT * FROM Purchasing.Vendor', adventureworks_conn)

In [508]:
adventureworks_sales_countryregioncurrency = pd.read_sql('SELECT * FROM Sales.CountryRegionCurrency', adventureworks_conn)
adventureworks_sales_creditcard = pd.read_sql('SELECT * FROM Sales.CreditCard', adventureworks_conn)
adventureworks_sales_currency = pd.read_sql('SELECT * FROM Sales.Currency', adventureworks_conn)
adventureworks_sales_currencyrate = pd.read_sql('SELECT * FROM Sales.CurrencyRate', adventureworks_conn)
adventureworks_sales_customer = pd.read_sql('SELECT * FROM Sales.Customer', adventureworks_conn)
adventureworks_sales_personcreditcard = pd.read_sql('SELECT * FROM Sales.PersonCreditCard', adventureworks_conn)
adventureworks_sales_salesorderdetail = pd.read_sql('SELECT * FROM Sales.SalesOrderDetail', adventureworks_conn)
adventureworks_sales_salesorderheader = pd.read_sql('SELECT * FROM Sales.SalesOrderHeader', adventureworks_conn)
adventureworks_sales_salesorderhearerrsaleseason = pd.read_sql('SELECT * FROM Sales.SalesOrderHeaderSalesReason', adventureworks_conn)
adventureworks_sales_salesperson = pd.read_sql('SELECT * FROM Sales.SalesPerson', adventureworks_conn)
adventureworks_sales_salespersonquotahistory = pd.read_sql('SELECT * FROM Sales.SalesPersonQuotaHistory', adventureworks_conn)
adventureworks_sales_salesreason = pd.read_sql('SELECT * FROM Sales.SalesReason', adventureworks_conn)
adventureworks_sales_salestaxrate = pd.read_sql('SELECT * FROM Sales.SalesTaxRate', adventureworks_conn)
adventureworks_sales_salesterritory = pd.read_sql('SELECT * FROM Sales.SalesTerritory', adventureworks_conn)
adventureworks_sales_salesterritoryhistory = pd.read_sql('SELECT * FROM Sales.SalesTerritoryHistory', adventureworks_conn)
adventureworks_sales_shoppingcartitem = pd.read_sql('SELECT * FROM Sales.ShoppingCartItem', adventureworks_conn)
adventureworks_sales_specialoffer = pd.read_sql('SELECT * FROM Sales.SpecialOffer', adventureworks_conn)
adventureworks_sales_specialofferproduct = pd.read_sql('SELECT * FROM Sales.SpecialOfferProduct', adventureworks_conn)
adventureworks_sales_store = pd.read_sql('SELECT * FROM Sales.Store', adventureworks_conn)

adventureworks_conn.close()

## Combining the data
ORDER MATTERS, CAUSE SURROGATE KEYS
SK STILL NEEDS TO BE DONE

### Departments
Combining aenc and adventureworks department data

In [509]:
# adding DEPARTMENT_source_database columns to the dataframes
aenc_department['DEPARTMENT_source_database'] = 'aenc'
adventureworks_humanresources_department['DEPARTMENT_source_database'] = 'adventureworks'

# combining all department data
departments = pd.concat([aenc_department, adventureworks_humanresources_department], ignore_index=True)

# combining name and department name columns to create a name column
departments['DEPARTMENT_DEPARTMENT_DeptName'] = departments['Name'].combine_first(departments['dept_name'])
# combining dept_id and DepartmentID columns
departments['DEPARTMENT_DEPARTMENT_DeptID'] = departments['dept_id'].combine_first(departments['DepartmentID'])

# dropping the redundant columns
drop_modified_date_rowguid(departments)
departments.drop(columns=['dept_id', 'Name', 'dept_name', 'DepartmentID'], inplace=True)

# renaming the remaining columns
departments.rename(columns={'dept_head_id': 'DEPARTMENT_DEPARTMENT_DeptHeadID', 'GroupName': 'DEPARTMENT_DEPARTMENT_GroupName'}, inplace=True)

departments.head()

C:\Users\yunyi\AppData\Local\Temp\ipykernel_43504\2444242683.py:11: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  departments['DEPARTMENT_DEPARTMENT_DeptID'] = departments['dept_id'].combine_first(departments['DepartmentID'])


,DEPARTMENT_DEPARTMENT_DeptHeadID,DEPARTMENT_source_database,DEPARTMENT_DEPARTMENT_GroupName,DEPARTMENT_DEPARTMENT_DeptName,DEPARTMENT_DEPARTMENT_DeptID
0,NaN,adventureworks,Research and Development,Engineering,1.0
1,NaN,adventureworks,Research and Development,Tool Design,2.0
2,NaN,adventureworks,Sales and Marketing,Sales,3.0
3,NaN,adventureworks,Sales and Marketing,Marketing,4.0
4,NaN,adventureworks,Inventory Management,Purchasing,5.0


### Employees

In [510]:
# merge northwind with territory and employee
employee1 = pd.merge(northwind_employees, northwind_employee_territories, on='EmployeeID')
employee1 = employee1.drop(columns=['EmployeeID', 'LastName', 'FirstName', 'BirthDate', 'Address', 'City', 'Region', 'PostalCode', 'Country', 'HomePhone', 'ReportsTo'])

# merge employee from aenc with bonus
employee2 = pd.merge(aenc_employee, aenc_bonus, on='emp_id')

# after that concat together
employees = pd.concat([employee1, employee2], ignore_index=True)

# rename
employees.rename(columns={
    'emp_id': 'EMPLOYEE_EMPLOYEE_EmployeeID', 
    'dept_id': 'EMPLOYEE_EMPLOYEE_DepartmentID', 
    'manager_id': 'EMPLOYEE_EMPLOYEE_ManagerID', 
    'territory_id' : 'EMPLOYEE_EMPLOYEETERRITORIES_TerritoryID',
    'emp_fname' : 'EMPLOYEE_EMPLOYEE_Emp_Fname',
    'emp_lname' : 'EMPLOYEE_EMPLOYEE_Emp_Lname',
    'street' : 'EMPLOYEE_EMPLOYEE_Street',
    'city' : 'EMPLOYEE_EMPLOYEE_City',
    'state' : 'EMPLOYEE_EMPLOYEE_State',
    'zip_code' : 'EMPLOYEE_EMPLOYEE_Zip_Code',
    'phone' : 'EMPLOYEE_EMPLOYEE_Phone',
    'status' : 'EMPLOYEE_EMPLOYEE_Status',
    'ss_number' : 'EMPLOYEE_EMPLOYEE_SS_Number',
    'salary' : 'EMPLOYEE_EMPLOYEE_Salary',
    'start_date' : 'EMPLOYEE_EMPLOYEE_Start_Date',
    'termination_date' : 'EMPLOYEE_EMPLOYEE_Termination',
    'birth_date' : 'EMPLOYEE_EMPLOYEE_Birth_Date',
    'bene_health_ins' : 'EMPLOYEE_EMPLOYEE_Bene_Health_Ins',
    'bene_life_ins' : 'EMPLOYEE_EMPLOYEE_Bene_Life_Ins',
    'bene_day_care' : 'EMPLOYEE_EMPLOYEE_Bene_Day_Care',
    'sex' : 'EMPLOYEE_EMPLOYEE_Sex',
    'bonus_date' : 'EMPLOYEE_BONUS_Bonus_Date',
    'bonus_amount' : 'EMPLOYEE_BONUS_Bonus_Amount',
    'Title' : 'EMPLOYEE_EMPLOYEES_Title',
    'TitleOfCourtesy' : 'EMPLOYEE_EMPLOYEES_TitleOfCourtesy',
    'HireDate' : 'EMPLOYEE_EMPLOYEES_HireDate',
    'HomePhone' : 'EMPLOYEE_EMPLOYEES_HomePhone',
    'Extension' : 'EMPLOYEE_EMPLOYEES_Extension',
    'PhotoPath' : 'EMPLOYEE_EMPLOYEES_PhotoPath',
    'Notes' : 'EMPLOYEE_EMPLOYEES_Notes'
}, inplace=True)

desired_columns_order = ['EMPLOYEE_EMPLOYEE_EmployeeID', 'EMPLOYEE_EMPLOYEE_DepartmentID', 'EMPLOYEE_EMPLOYEE_ManagerID', 'EMPLOYEE_EMPLOYEETERRITORIES_TerritoryID','EMPLOYEE_EMPLOYEE_Emp_Fname','EMPLOYEE_EMPLOYEE_Emp_Lname','EMPLOYEE_EMPLOYEE_Street','EMPLOYEE_EMPLOYEE_City','EMPLOYEE_EMPLOYEE_State','EMPLOYEE_EMPLOYEE_Zip_Code','EMPLOYEE_EMPLOYEE_Phone','EMPLOYEE_EMPLOYEE_Status','EMPLOYEE_EMPLOYEE_SS_Number','EMPLOYEE_EMPLOYEE_Salary','EMPLOYEE_EMPLOYEE_Start_Date','EMPLOYEE_EMPLOYEE_Termination','EMPLOYEE_EMPLOYEE_Birth_Date','EMPLOYEE_EMPLOYEE_Bene_Health_Ins','EMPLOYEE_EMPLOYEE_Bene_Life_Ins','EMPLOYEE_EMPLOYEE_Bene_Day_Care','EMPLOYEE_EMPLOYEE_Sex','EMPLOYEE_BONUS_Bonus_Date','EMPLOYEE_BONUS_Bonus_Amount','EMPLOYEE_EMPLOYEES_Title','EMPLOYEE_EMPLOYEES_TitleOfCourtesy','EMPLOYEE_EMPLOYEES_HireDate','EMPLOYEE_EMPLOYEES_HomePhone','EMPLOYEE_EMPLOYEES_Extension','EMPLOYEE_EMPLOYEES_PhotoPath','EMPLOYEE_EMPLOYEES_Notes', 'EMPLOYEE_EMPLOYEES_PhotoHexString']

#order
employees = employees.reindex(columns=desired_columns_order)

employees.head()

,EMPLOYEE_EMPLOYEE_EmployeeID,EMPLOYEE_EMPLOYEE_DepartmentID,EMPLOYEE_EMPLOYEE_ManagerID,EMPLOYEE_EMPLOYEETERRITORIES_TerritoryID,EMPLOYEE_EMPLOYEE_Emp_Fname,EMPLOYEE_EMPLOYEE_Emp_Lname,EMPLOYEE_EMPLOYEE_Street,EMPLOYEE_EMPLOYEE_City,EMPLOYEE_EMPLOYEE_State,EMPLOYEE_EMPLOYEE_Zip_Code,EMPLOYEE_EMPLOYEE_Phone,EMPLOYEE_EMPLOYEE_Status,EMPLOYEE_EMPLOYEE_SS_Number,EMPLOYEE_EMPLOYEE_Salary,EMPLOYEE_EMPLOYEE_Start_Date,EMPLOYEE_EMPLOYEE_Termination,EMPLOYEE_EMPLOYEE_Birth_Date,EMPLOYEE_EMPLOYEE_Bene_Health_Ins,EMPLOYEE_EMPLOYEE_Bene_Life_Ins,EMPLOYEE_EMPLOYEE_Bene_Day_Care,EMPLOYEE_EMPLOYEE_Sex,EMPLOYEE_BONUS_Bonus_Date,EMPLOYEE_BONUS_Bonus_Amount,EMPLOYEE_EMPLOYEES_Title,EMPLOYEE_EMPLOYEES_TitleOfCourtesy,EMPLOYEE_EMPLOYEES_HireDate,EMPLOYEE_EMPLOYEES_HomePhone,EMPLOYEE_EMPLOYEES_Extension,EMPLOYEE_EMPLOYEES_PhotoPath,EMPLOYEE_EMPLOYEES_Notes,EMPLOYEE_EMPLOYEES_PhotoHexString
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sales Representative,Ms.,1992-05-01,NaN,5467,http://accweb/emmployees/davolio.bmp,Education includes a BA in psychology from Col...,0x151C2F00020000000D000E0014002100FFFFFFFF4269...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sales Representative,Ms.,1992-05-01,NaN,5467,http://accweb/emmployees/davolio.bmp,Education includes a BA in psychology from Col...,0x151C2F00020000000D000E0014002100FFFFFFFF4269...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Vice President, Sales",Dr.,1992-08-14,NaN,3457,http://accweb/emmployees/fuller.bmp,Andrew received his BTS commercial in 1974 and...,0x151C2F00020000000D000E0014002100FFFFFFFF4269...
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Vice President, Sales",Dr.,1992-08-14,NaN,3457,http://accweb/emmployees/fuller.bmp,Andrew received his BTS commercial in 1974 and...,0x151C2F00020000000D000E0014002100FFFFFFFF4269...
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Vice President, Sales",Dr.,1992-08-14,NaN,3457,http://accweb/emmployees/fuller.bmp,Andrew received his BTS commercial in 1974 and...,0x151C2F00020000000D000E0014002100FFFFFFFF4269...


### BusinessEntities

In [511]:
# Combining the adventureworks Person.BusinessEntity, BusinessEntityContact and ContactType data
businessentities= pd.merge(adventureworks_person_businessentity, adventureworks_person_businessentitycontact, left_on='BusinessEntityID', right_on='BusinessEntityID', suffixes=('_person_businessentity', '_businessentitycontact'), how="outer")

businessentities = pd.merge(businessentities, adventureworks_person_contacttype, left_on='ContactTypeID', right_on='ContactTypeID', suffixes=('', '_contacttype'), how="outer")

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(businessentities)

# renaming the columns
businessentities.rename(columns={'BusinessEntityID': 'BUSINESSENTITY_BUSINESSENTITY_BusinessEntityID', 'PersonID': 'BUSINESSENTITY_BUSINESSENTITYCONTACT_PersonID', 'ContactTypeID': 'BUSINESSENTITY_CONTACTTYPE_ContactTypeID', 'Name' : 'BUSINESSENTITY_CONTACTTYPE_Name'}, inplace=True)

businessentities.head()

,BUSINESSENTITY_BUSINESSENTITY_BusinessEntityID,BUSINESSENTITY_BUSINESSENTITYCONTACT_PersonID,BUSINESSENTITY_CONTACTTYPE_ContactTypeID,BUSINESSENTITY_CONTACTTYPE_Name
0,NaN,NaN,1.0,Accounting Manager
1,1510.0,1509.0,2.0,Assistant Sales Agent
2,1518.0,1517.0,2.0,Assistant Sales Agent
3,1522.0,1521.0,2.0,Assistant Sales Agent
4,1528.0,1527.0,2.0,Assistant Sales Agent


### BillOfMaterial

In [512]:
# merge table
billofmaterials = pd.merge(adventureworks_production_bill_of_materials, adventureworks_production_unitmeasure, on='UnitMeasureCode')

# rename
billofmaterials.rename(columns={
    'BillOfMaterialsID': 'BILLOFMATERIAL_BILLOFMATERIAL_BillOfMaterialID',
    'ProductAssemblyID': 'BILLOFMATERIAL_BILLOFMATERIAL_ProductAssemblyID',
    'ComponentID': 'BILLOFMATERIAL_BILLOFMATERIAL_ComponentID',
    'StartDate': 'BILLOFMATERIAL_BILLOFMATERIAL_StartDate',
    'EndDate': 'BILLOFMATERIAL_BILLOFMATERIAL_EndDate',
    'UnitMeasureCode': 'BILLOFMATERIAL_BILLOFMATERIAL_UnitMeasureCode',
    'BOMLevel': 'BILLOFMATERIAL_BILLOFMATERIAL_BOMLevel',
    'PerAssemblyQty': 'BILLOFMATERIAL_BILLOFMATERIAL_PerAssemblyQty',
    'Name': 'BILLOFMATERIAL_UNITMEASURE_Name'
}, inplace=True)

# drop unneeded column
drop_modified_date_rowguid(billofmaterials)

billofmaterials.head()

,BILLOFMATERIAL_BILLOFMATERIAL_BillOfMaterialID,BILLOFMATERIAL_BILLOFMATERIAL_ProductAssemblyID,BILLOFMATERIAL_BILLOFMATERIAL_ComponentID,BILLOFMATERIAL_BILLOFMATERIAL_StartDate,BILLOFMATERIAL_BILLOFMATERIAL_EndDate,BILLOFMATERIAL_BILLOFMATERIAL_UnitMeasureCode,BILLOFMATERIAL_BILLOFMATERIAL_BOMLevel,BILLOFMATERIAL_BILLOFMATERIAL_PerAssemblyQty,BILLOFMATERIAL_UNITMEASURE_Name
0,893,NaN,749,2010-05-26,NaT,EA,0,1.0,Each
1,271,NaN,750,2010-03-04,2010-05-03,EA,0,1.0,Each
2,34,NaN,750,2010-05-04,NaT,EA,0,1.0,Each
3,830,NaN,751,2010-05-26,NaT,EA,0,1.0,Each
4,2074,NaN,752,2010-07-08,NaT,EA,0,1.0,Each


### Document

In [513]:
# merge table
documents = pd.merge(adventureworks_production_document, adventureworks_production_productdocument, on='DocumentNode')

# rename
documents.rename(columns={
    'DocumentNode': 'DOCUMENT_DOCUMENT_DocumentNode',
    'DocumentLevel': 'DOCUMENT_DOCUMENT_DocumentLevel',
    'Title': 'DOCUMENT_DOCUMENT_Title',
    'Owner': 'DOCUMENT_DOCUMENT_Owner',
    'FolderFlag': 'DOCUMENT_DOCUMENT_FolderFlag',
    'FileName': 'DOCUMENT_DOCUMENT_FileName',
    'FileExtension': 'DOCUMENT_DOCUMENT_FileExtension',
    'Revision': 'DOCUMENT_DOCUMENT_Revision',
    'ChangeNumber': 'DOCUMENT_DOCUMENT_ChangeNumber',
    'Status': 'DOCUMENT_DOCUMENT_Status',
    'DocumentSummary': 'DOCUMENT_DOCUMENT_DocumentSummary',
    'Document': 'DOCUMENT_DOCUMENT_Document',
    'ProductID': 'DOCUMENT_PRODUCTDOCUMENT_ProductID'
}, inplace=True)

# drop unneeded column
drop_modified_date_rowguid(documents)
documents.drop(columns=['DocumentNodeString'], inplace=True)

documents.head()

,DOCUMENT_DOCUMENT_DocumentNode,DOCUMENT_DOCUMENT_DocumentLevel,DOCUMENT_DOCUMENT_Title,DOCUMENT_DOCUMENT_Owner,DOCUMENT_DOCUMENT_FolderFlag,DOCUMENT_DOCUMENT_FileName,DOCUMENT_DOCUMENT_FileExtension,DOCUMENT_DOCUMENT_Revision,DOCUMENT_DOCUMENT_ChangeNumber,DOCUMENT_DOCUMENT_Status,DOCUMENT_DOCUMENT_DocumentSummary,DOCUMENT_DOCUMENT_Document,DOCUMENT_PRODUCTDOCUMENT_ProductID
0,b'Z\xc0',2,Introduction 1,219,False,Introduction 1.doc,.doc,4,28,2,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,977
1,b'Z\xc0',2,Introduction 1,219,False,Introduction 1.doc,.doc,4,28,2,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,997
2,b'Z\xc0',2,Introduction 1,219,False,Introduction 1.doc,.doc,4,28,2,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,998
3,b'Z\xc0',2,Introduction 1,219,False,Introduction 1.doc,.doc,4,28,2,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,999
4,b'j\xc0',2,Crank Arm and Tire Maintenance,219,False,Crank Arm and Tire Maintenance.doc,.doc,0,0,1,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,317


### WorkOrder

In [514]:
# drop product id
adventureworks_production_workorderrouting = adventureworks_production_workorderrouting.drop(columns=('ProductID'))

#merge table
workorders = pd.merge(adventureworks_production_workorder, adventureworks_production_scrapreason, on="ScrapReasonID")
workorders = pd.merge(workorders, adventureworks_production_workorderrouting, on="WorkOrderID")

#rename
workorders.rename(columns={
    'WorkOrderID': 'WORKORDER_WORKORDER_WorkOrderID',
    'ProductID': 'WORKORDER_WORKORDER_ProductID',
    'OrderQty': 'WORKORDER_WORKORDER_OrderQty',
    'StockedQty': 'WORKORDER_WORKORDER_StockedQty',
    'ScrappedQty': 'WORKORDER_WORKORDER_ScrappedQty',
    'StartDate': 'WORKORDER_WORKORDER_StartDate',
    'EndDate': 'WORKORDER_WORKORDER_EndDate',
    'DueDate': 'WORKORDER_WORKORDER_DueDate',
    'ScrapReasonID': 'WORKORDER_SCRAPREASON_ScrapReasonID',
    'Name': 'WORKORDER_SCRAPREASON_Name',
    'OperationSequence': 'WORKORDER_WORKORDERINGROUTING_OperationSequence',
    'LocationID': 'WORKORDER_WORKORDERINGROUTING_LocationID',
    'ScheduledStartDate': 'WORKORDER_WORKORDERINGROUTING_ScheduledStartDate',
    'ScheduledEndDate': 'WORKORDER_WORKORDERINGROUTING_ScheduledEndDate',
    'ActualStartDate': 'WORKORDER_WORKORDERINGROUTING_ActualStartDate',
    'ActualEndDate': 'WORKORDER_WORKORDERINGROUTING_ActualEndDate',
    'ActualResourceHrs': 'WORKORDER_WORKORDERINGROUTING_ActualResourcesHrs',
    'PlannedCost': 'WORKORDER_WORKORDERINGROUTING_PlannedCost',
    'ActualCost': 'WORKORDER_WORKORDERINGROUTING_ActualCost'
}, inplace=True)

#drop unneeded column
drop_modified_date_rowguid(workorders)

workorders.head()

,WORKORDER_WORKORDER_WorkOrderID,WORKORDER_WORKORDER_ProductID,WORKORDER_WORKORDER_OrderQty,WORKORDER_WORKORDER_StockedQty,WORKORDER_WORKORDER_ScrappedQty,WORKORDER_WORKORDER_StartDate,WORKORDER_WORKORDER_EndDate,WORKORDER_WORKORDER_DueDate,WORKORDER_SCRAPREASON_ScrapReasonID,WORKORDER_SCRAPREASON_Name,WORKORDER_WORKORDERINGROUTING_OperationSequence,WORKORDER_WORKORDERINGROUTING_LocationID,WORKORDER_WORKORDERINGROUTING_ScheduledStartDate,WORKORDER_WORKORDERINGROUTING_ScheduledEndDate,WORKORDER_WORKORDERINGROUTING_ActualStartDate,WORKORDER_WORKORDERINGROUTING_ActualEndDate,WORKORDER_WORKORDERINGROUTING_ActualResourcesHrs,WORKORDER_WORKORDERINGROUTING_PlannedCost,WORKORDER_WORKORDERINGROUTING_ActualCost
0,41,518,98,97,1,2011-06-03,2011-06-19,2011-06-14,7.0,Handling damage,6,50,2011-06-03,2011-06-14,2011-06-03,2011-06-19,3.0,36.75,36.75
1,41,518,98,97,1,2011-06-03,2011-06-19,2011-06-14,7.0,Handling damage,7,60,2011-06-03,2011-06-14,2011-06-03,2011-06-19,4.0,49.00,49.00
2,69,810,120,117,3,2011-06-03,2011-06-19,2011-06-14,11.0,Stress test failed,1,10,2011-06-03,2011-06-14,2011-06-03,2011-06-19,4.1,92.25,92.25
3,69,810,120,117,3,2011-06-03,2011-06-19,2011-06-14,11.0,Stress test failed,2,20,2011-06-03,2011-06-14,2011-06-03,2011-06-19,3.5,87.50,87.50
4,69,810,120,117,3,2011-06-03,2011-06-19,2011-06-14,11.0,Stress test failed,6,50,2011-06-03,2011-06-14,2011-06-03,2011-06-19,3.0,36.75,36.75


### BusinessEntityAddresses

In [515]:
# Combining the adventureworks Person.BusinessEntityAddress, Address and AddressType data
businessentityaddresses = pd.merge(adventureworks_person_businessentityaddress, adventureworks_person_address, left_on='AddressID', right_on='AddressID', suffixes=('', '_address'), how="outer")

businessentityaddresses = pd.merge(businessentityaddresses, adventureworks_person_address_type, left_on='AddressTypeID', right_on='AddressTypeID', suffixes=('', '_address_type'), how="outer")


# dropping the modified date and rowguid columns
drop_modified_date_rowguid(businessentityaddresses)

# renaming the columns
businessentityaddresses.rename(columns={'BusinessEntityID': 'BUSINESSENTITYADDRESS_BUSINESSENTITYADDRESS_BusinessEntityID', 'AddressID': 'BUSINESSENTITYADDRESS_ADDRESS_AddressID', 'AddressTypeID': 'BUSINESSENTITYADDRESS_ADDRESSTYPE_AddressTypeID', 'AddressLine1' : 'BUSINESSENTITYADDRESS_ADDRESS_AddressLine1', 'AddressLine2' : 'BUSINESSENTITYADDRESS_ADDRESS_AddressLine2', 'City' : 'BUSINESSENTITYADDRESS_ADDRESS_City', 'StateProvinceID' : 'BUSINESSENTITYADDRESS_ADDRESS_StateProvinceID', 'PostalCode' : 'BUSINESSENTITYADDRESS_ADDRESS_POSTALCODE', 'SpatialLocation' : 'BUSINESSENTITYADDRESS_ADDRESS_SpatialLocation', 'Name' : 'BUSINESSENTITYADDRESS_ADDRESSTYPE_Name'}, inplace=True)

businessentityaddresses.head()

,BUSINESSENTITYADDRESS_BUSINESSENTITYADDRESS_BusinessEntityID,BUSINESSENTITYADDRESS_ADDRESS_AddressID,BUSINESSENTITYADDRESS_ADDRESSTYPE_AddressTypeID,BUSINESSENTITYADDRESS_ADDRESS_AddressLine1,BUSINESSENTITYADDRESS_ADDRESS_AddressLine2,BUSINESSENTITYADDRESS_ADDRESS_City,BUSINESSENTITYADDRESS_ADDRESS_StateProvinceID,BUSINESSENTITYADDRESS_ADDRESS_POSTALCODE,BUSINESSENTITYADDRESS_ADDRESS_SpatialLocation,BUSINESSENTITYADDRESS_ADDRESSTYPE_Name
0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,Billing
1,12.0,1.0,2,1970 Napa Ct.,None,Bothell,79.0,98011,POINT (-122.164644615406 47.7869921906598),Home
2,123.0,2.0,2,9833 Mt. Dias Blv.,None,Bothell,79.0,98011,POINT (-122.250185528911 47.6867097047995),Home
3,285.0,3.0,2,7484 Roundtree Drive,None,Bothell,79.0,98011,POINT (-122.274625789912 47.7631154083121),Home
4,251.0,4.0,2,9539 Glenside Dr,None,Bothell,79.0,98011,POINT (-122.335726442416 47.7392386259644),Home


### People

In [516]:
# combining person data from adventureworks HumanResources.Person, PersonPhone, PhoneNumberType, EmailAddress and Password
people = pd.merge(adventureworks_person_person, adventureworks_person_personphone, left_on='BusinessEntityID', right_on='BusinessEntityID', suffixes= ('_person', '_personphone'), how="outer")

people = pd.merge(people, adventureworks_person_phonenumbertype, left_on='PhoneNumberTypeID', right_on='PhoneNumberTypeID', suffixes=('', '_phonenumbertype'), how="outer")

people = pd.merge(people, adventureworks_person_emailaddress, left_on='BusinessEntityID', right_on='BusinessEntityID', suffixes=('','_emailaddress'), how="outer")

people = pd.merge(people, adventureworks_person_password, left_on='BusinessEntityID', right_on='BusinessEntityID', suffixes=('','_password'), how="outer")

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(people)

# renaming the columns
people.rename(columns={'BusinessEntityID': 'PERSON_PERSON_BusinessEntityID', 'PersonType': 'PERSON_PERSON_PersonType', 'NameStyle': 'PERSON_PERSON_NameStyle', 'Title': 'PERSON_PERSON_Title', 'FirstName': 'PERSON_PERSON_FirstName', 'MiddleName' : 'PERSON_PERSON_MiddleName', 'LastName' : 'PERSON_PERSON_LastName', 'Suffix': 'PERSON_PERSON_Suffix', 'EmailPromotion' : 'PERSON_PERSON_EmailPromotion', 'AdditionalContactInfo' : 'PERSON_PERSON_AdditionalContactInfo', 'Demographics' : 'PERSON_PERSON_Demographics', 'PhoneNumber' : 'PERSON_PERSONPHONE_PhoneNumber', 'PhoneNumberTypeID' : 'PERSON_PHONENUMBERTYPE_PhoneNumberTypeID', 'Name' : 'PERSON_PHONENUMBERTYPE_Name', 'EmailAddressID': 'PERSON_EMAILADDRESS_EmailAddressID', 'EmailAddress' : 'PERSON_EMAILADDRESS_EmailAddress', 'PasswordHash' : 'PERSON_PASSWORD_PasswordHash', 'PasswordSalt' : 'PERSON_PASSWORD_PasswordSalt'}, inplace=True)

people.head()

,PERSON_PERSON_BusinessEntityID,PERSON_PERSON_PersonType,PERSON_PERSON_NameStyle,PERSON_PERSON_Title,PERSON_PERSON_FirstName,PERSON_PERSON_MiddleName,PERSON_PERSON_LastName,PERSON_PERSON_Suffix,PERSON_PERSON_EmailPromotion,PERSON_PERSON_AdditionalContactInfo,PERSON_PERSON_Demographics,PERSON_PERSONPHONE_PhoneNumber,PERSON_PHONENUMBERTYPE_PhoneNumberTypeID,PERSON_PHONENUMBERTYPE_Name,PERSON_EMAILADDRESS_EmailAddressID,PERSON_EMAILADDRESS_EmailAddress,PERSON_PASSWORD_PasswordHash,PERSON_PASSWORD_PasswordSalt
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",697-555-0142,1,Cell,1,ken0@adventure-works.com,pbFwXWE99vobT6g+vPWFy93NtUU/orrIWafF01hccfM=,bE3XiWw=
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",819-555-0175,3,Work,2,terri0@adventure-works.com,bawRVNrZQYQ05qF05Gz6VLilnviZmrqBReTTAGAudm0=,EjJaC3U=
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",212-555-0187,1,Cell,3,roberto0@adventure-works.com,8BUXrZfDqO1IyHCWOYzYmqN1IhTUn3CJMpdx/UCQ3iY=,wbPZqMw=
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",612-555-0100,1,Cell,4,rob0@adventure-works.com,SjLXpiarHSlz+6AG+H+4QpB/IPRzras/+9q/5Wr7tf8=,PwSunQU=
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",849-555-0139,1,Cell,5,gail0@adventure-works.com,8FYdAiY6gWuBsgjCFdg0UibtsqOcWHf9TyaHIP7+paA=,qYhZRiM=


### JobCandidate

In [517]:
jobcandidates = adventureworks_humanresources_jobcandidate

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(jobcandidates)

# renaming the columns
jobcandidates.rename(columns={'JobCandidateID': 'JOBCANDIDATE_JOBCANDIDATE_JobCandidateID', 'BusinessEntityID': 'JOBCANDIDATE_JOBCANDIDATE_BusinessEntityID', 'Resume': 'JOBCANDIDATE_JOBCANDIDATE_Resume'}, inplace=True)

jobcandidates.head()

,JOBCANDIDATE_JOBCANDIDATE_JobCandidateID,JOBCANDIDATE_JOBCANDIDATE_BusinessEntityID,JOBCANDIDATE_JOBCANDIDATE_Resume
0,1,NaN,"<ns:Resume xmlns:ns=""http://schemas.microsoft...."
1,2,NaN,"<ns:Resume xmlns:ns=""http://schemas.microsoft...."
2,3,NaN,"<ns:Resume xmlns:ns=""http://schemas.microsoft...."
3,4,274.0,"<ns:Resume xmlns:ns=""http://schemas.microsoft...."
4,5,NaN,"<ns:Resume xmlns:ns=""http://schemas.microsoft...."


### EmployeeDepartmentHistory

In [518]:
employeedepartmenthistories = adventureworks_humanresources_employeedepartmenthistory

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(employeedepartmenthistories)

# renaming the columns
employeedepartmenthistories.rename(columns={'BusinessEntityID': 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_BusinessEntityID', 'DepartmentID': 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_DepartmentID', 'ShiftID': 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_ShiftID', 'StartDate': 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_StartDate', 'EndDate': 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_EndDate'}, inplace=True)

employeedepartmenthistories.head()

,EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_BusinessEntityID,EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_DepartmentID,EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_ShiftID,EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_StartDate,EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_EndDate
0,1,16,1,2009-01-14,None
1,2,1,1,2008-01-31,None
2,3,1,1,2007-11-11,None
3,4,1,1,2007-12-05,2010-05-30
4,4,2,1,2010-05-31,None


### EmployeePayHistory

In [519]:
employeepayhistories = adventureworks_humanresources_employeepayhistory

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(employeepayhistories)

# renaming the columns
employeepayhistories.rename(columns={'BusinessEntityID': 'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_BusinessEntityID', 'RateChangeDate': 'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_RateChangeDate', 'Rate': 'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_Rate', 'PayFrequency': 'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_PayFrequency'}, inplace=True)

employeepayhistories.head()

,EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_BusinessEntityID,EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_RateChangeDate,EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_Rate,EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_PayFrequency
0,1,2009-01-14,125.5000,2
1,2,2008-01-31,63.4615,2
2,3,2007-11-11,43.2692,2
3,4,2007-12-05,8.6200,2
4,4,2010-05-31,23.7200,2


### Shift

In [520]:
shifts = adventureworks_humanresources_shift

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(shifts)

# renaming the columns
shifts.rename(columns={'ShiftID': 'SHIFT_SHIFT_ShiftID', 'Name': 'SHIFT_SHIFT_Name', 'StartTime': 'SHIFT_SHIFT_StartTime', 'EndTime': 'SHIFT_SHIFT_EndTime'}, inplace=True)

shifts.head()

,SHIFT_SHIFT_ShiftID,SHIFT_SHIFT_Name,SHIFT_SHIFT_StartTime,SHIFT_SHIFT_EndTime
0,1,Day,07:00:00.0000000,15:00:00.0000000
1,2,Evening,15:00:00.0000000,23:00:00.0000000
2,3,Night,23:00:00.0000000,07:00:00.0000000


### SalesPerson

In [521]:
salespeople = adventureworks_sales_salesperson

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(salespeople)

# renaming the columns
salespeople.rename(columns={'BusinessEntityID': 'SALESPERSON_SALESPERSON_BusinessEntityID', 'TerritoryID': 'SALESPERSON_SALESPERSON_TerritoryID', 'SalesQuota': 'SALESPERSON_SALESPERSON_SalesQuota', 'Bonus': 'SALESPERSON_SALESPERSON_Bonus', 'CommissionPct': 'SALESPERSON_SALESPERSON_CommissionPct', 'SalesYTD': 'SALESPERSON_SALESPERSON_SalesYTD', 'SalesLastYear': 'SALESPERSON_SALESPERSON_SalesLastYear'}, inplace=True)

salespeople.head()

,SALESPERSON_SALESPERSON_BusinessEntityID,SALESPERSON_SALESPERSON_TerritoryID,SALESPERSON_SALESPERSON_SalesQuota,SALESPERSON_SALESPERSON_Bonus,SALESPERSON_SALESPERSON_CommissionPct,SALESPERSON_SALESPERSON_SalesYTD,SALESPERSON_SALESPERSON_SalesLastYear
0,274,NaN,NaN,0.0,0.000,5.596976e+05,0.000000e+00
1,275,2.0,300000.0,4100.0,0.012,3.763178e+06,1.750406e+06
2,276,4.0,250000.0,2000.0,0.015,4.251369e+06,1.439156e+06
3,277,3.0,250000.0,2500.0,0.015,3.189418e+06,1.997186e+06
4,278,6.0,250000.0,500.0,0.010,1.453719e+06,1.620277e+06


### ProductVendor

In [522]:
productvendors = adventureworks_purchasing_productvendor

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(productvendors)

# renaming the columns
productvendors.rename(columns={'ProductID': 'PRODUCTVENDOR_PRODUCTVENDOR_ProductID', 'BusinessEntityID': 'PRODUCTVENDOR_PRODUCTVENDOR_BusinessEntityID', 'AverageLeadTime': 'PRODUCTVENDOR_PRODUCTVENDOR_AverageLeadTime', 'StandardPrice': 'PRODUCTVENDOR_PRODUCTVENDOR_StandardPrice', 'LastReceiptCost': 'PRODUCTVENDOR_PRODUCTVENDOR_LastReceiptCost', 'LastReceiptDate': 'PRODUCTVENDOR_PRODUCTVENDOR_LastReceiptDate', 'MinOrderQty': 'PRODUCTVENDOR_PRODUCTVENDOR_MinOrderQty', 'MaxOrderQty': 'PRODUCTVENDOR_PRODUCTVENDOR_MaxOrderQty', 'OnOrderQty': 'PRODUCTVENDOR_PRODUCTVENDOR_OnOrderQty', 'UnitMeasureCode' : 'PRODUCTVENDOR_PRODUCTVENDOR_UnitMeasureCode'}, inplace=True)

productvendors.head()

,PRODUCTVENDOR_PRODUCTVENDOR_ProductID,PRODUCTVENDOR_PRODUCTVENDOR_BusinessEntityID,PRODUCTVENDOR_PRODUCTVENDOR_AverageLeadTime,PRODUCTVENDOR_PRODUCTVENDOR_StandardPrice,PRODUCTVENDOR_PRODUCTVENDOR_LastReceiptCost,PRODUCTVENDOR_PRODUCTVENDOR_LastReceiptDate,PRODUCTVENDOR_PRODUCTVENDOR_MinOrderQty,PRODUCTVENDOR_PRODUCTVENDOR_MaxOrderQty,PRODUCTVENDOR_PRODUCTVENDOR_OnOrderQty,PRODUCTVENDOR_PRODUCTVENDOR_UnitMeasureCode
0,1,1580,17,47.87,50.2635,2011-08-29,1,5,3.0,CS
1,2,1688,19,39.92,41.9160,2011-08-29,1,5,3.0,CTN
2,4,1650,17,54.31,57.0255,2011-08-29,1,5,NaN,CTN
3,317,1578,19,28.17,29.5785,2011-08-29,100,1000,300.0,EA
4,317,1678,17,25.77,27.0585,2011-08-25,100,1000,NaN,EA


### CustomerCustomerDemo

In [523]:
customercustomerdemos = northwind_customer_customer_demo

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(customercustomerdemos)

# renaming the columns
customercustomerdemos.rename(columns={'CustomerID': 'CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerID', 'CustomerTypeID': 'CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerTypeID'}, inplace=True)

customercustomerdemos.head()

,CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerID,CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerTypeID


### CustomerDemographics

In [524]:
customerdemographics = northwind_customer_demographics

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(customerdemographics)

# renaming the columns
customerdemographics.rename(columns={'CustomerTypeID': 'CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerTypeID', 'CustomerDesc': 'CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerDesc'}, inplace=True)

customerdemographics.head()

,CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerTypeID,CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerDesc


### SalesTerritoryHistory

In [525]:
salesterritoryhistories = adventureworks_sales_salesterritoryhistory

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(salesterritoryhistories)

# renaming the columns
salesterritoryhistories.rename(columns={'BusinessEntityID': 'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_BusinessEntityID', 'TerritoryID': 'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_TerritoryID', 'StartDate': 'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_StartDate', 'EndDate': 'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_EndDate'}, inplace=True)

salesterritoryhistories.head()

,SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_BusinessEntityID,SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_TerritoryID,SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_StartDate,SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_EndDate
0,275,2,2011-05-31,2012-11-29
1,275,3,2012-11-30,NaT
2,276,4,2011-05-31,NaT
3,277,3,2011-05-31,2012-11-29
4,277,2,2012-11-30,NaT


### ProductListPriceHistory

In [526]:
productlistpricehistories = adventureworks_production_productlistpricehistory

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(productlistpricehistories)

# renaming the columns
productlistpricehistories.rename(columns={'ProductID': 'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ProductID', 'StartDate': 'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_StartDate', 'EndDate': 'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_EndDate', 'ListPrice': 'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ListPrice'}, inplace=True)

productlistpricehistories.head()

,PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ProductID,PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_StartDate,PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_EndDate,PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ListPrice
0,707,2011-05-31,2012-05-29,33.6442
1,707,2012-05-30,2013-05-29,33.6442
2,707,2013-05-30,NaT,34.9900
3,708,2011-05-31,2012-05-29,33.6442
4,708,2012-05-30,2013-05-29,33.6442


### ProductCostHistory

In [527]:
productcosthistories = adventureworks_production_productcosthistory

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(productcosthistories)

# renaming the columns
productcosthistories.rename(columns={'ProductID': 'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_ProductID', 'StartDate': 'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_StartDate', 'EndDate': 'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_EndDate', 'StandardCost': 'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_StandardCost'}, inplace=True)

productcosthistories.head()

,PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_ProductID,PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_StartDate,PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_EndDate,PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_StandardCost
0,707,2011-05-31,2012-05-29,12.0278
1,707,2012-05-30,2013-05-29,13.8782
2,707,2013-05-30,NaT,13.0863
3,708,2011-05-31,2012-05-29,12.0278
4,708,2012-05-30,2013-05-29,13.8782


### ShoppingCartItem

In [528]:
shoppingcartitems = adventureworks_sales_shoppingcartitem

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(shoppingcartitems)

# renaming the columns
shoppingcartitems.rename(columns={'ShoppingCartItemID': 'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartItemID', 'ShoppingCartID': 'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartID', 'Quantity': 'SHOPPINGCARTITEM_SHOPPINGCARTITEM_Quantity', 'ProductID': 'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ProductID', 'DateCreated': 'SHOPPINGCARTITEM_SHOPPINGCARTITEM_DateCreated'}, inplace=True)

shoppingcartitems.head()

,SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartItemID,SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartID,SHOPPINGCARTITEM_SHOPPINGCARTITEM_Quantity,SHOPPINGCARTITEM_SHOPPINGCARTITEM_ProductID,SHOPPINGCARTITEM_SHOPPINGCARTITEM_DateCreated
0,2,14951,3,862,2013-11-09 17:54:07.603
1,4,20621,4,881,2013-11-09 17:54:07.603
2,5,20621,7,874,2013-11-09 17:54:07.603


### SalesPersonQuotaHistory

In [529]:
salespersonquotahistories = adventureworks_sales_salespersonquotahistory

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(salespersonquotahistories)

# renaming the columns
salespersonquotahistories.rename(columns={'BusinessEntityID': 'SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_BusinessEntityID', 'QuotaDate': 'SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_QuotaDate', 'SalesQuota': 'SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_SalesQuota'}, inplace=True)

salespersonquotahistories.head()

,SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_BusinessEntityID,SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_QuotaDate,SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_SalesQuota
0,274,2011-05-31,28000.0
1,274,2011-08-31,7000.0
2,274,2011-12-01,91000.0
3,274,2012-02-29,140000.0
4,274,2012-05-30,70000.0


### Customer

In [530]:
aenc_customer = aenc_customer[['fname', 'lname', 'state']]
northwind_customers.drop(columns='CustomerID', inplace=True)

# combining all customer data
customers = pd.concat([northwind_customers, aenc_customer, adventureworks_sales_customer], ignore_index=True)

#combine with store
customers = pd.merge(customers, adventureworks_sales_store, left_on='StoreID', right_on='BusinessEntityID')
customers.drop(columns='BusinessEntityID', inplace=True)

# combine with sales territory
customers = pd.merge(customers, adventureworks_sales_salesterritory, on='TerritoryID')

# rename
customers.rename(columns={
    'CustomerID': 'CUSTOMER_CUSTOMERS_ID',
    'CompanyName': 'CUSTOMER_CUSTOMERS_CompanyName',
    'ContactName': 'CUSTOMER_CUSTOMERS_ContactName',
    'ContactTitle': 'CUSTOMER_CUSTOMERS_ContactTitle',
    'Address': 'CUSTOMER_CUSTOMERS_Address',
    'City': 'CUSTOMER_CUSTOMERS_City',
    'Region': 'CUSTOMER_CUSTOMERS_Region',
    'PostalCode': 'CUSTOMER_CUSTOMERS_PostalCode',
    'Country': 'CUSTOMER_CUSTOMERS_Country',
    'Phone': 'CUSTOMER_CUSTOMERS_Phone',
    'Fax': 'CUSTOMER_CUSTOMERS_Fax',
    'fname': 'CUSTOMER_CUSTOMER_Fname',
    'lname': 'CUSTOMER_CUSTOMER_Lname',
    'state': 'CUSTOMER_CUSTOMER_State',
    'PersonID': 'CUSTOMER_CUSTOMER_PersonID',
    'StoreID': 'CUSTOMER_CUSTOMER_StoreID',
    'TerritoryID': 'CUSTOMER_CUSTOMER_TerritoryID',
    'AccountNumber': 'CUSTOMER_CUSTOMER_AccountNumber',
    'Name_x': 'CUSTOMER_STORE_Name',
    'SalesPersonID': 'CUSTOMER_STORE_SalesPersonID',
    'Demographics': 'CUSTOMER_STORE_Demographics',
    'Name_y': 'CUSTOMER_SALESTERRITORY_Name',
    'CountryRegionCode': 'CUSTOMER_SALESTERRITORY_CountryRegionCode',
    'Group': 'CUSTOMER_SALESTERRITORY_Group',
    'SalesYTD': 'CUSTOMER_SALESTERRITORY_SalesYTD',
    'SalesLastYear': 'CUSTOMER_SALESTERRITORY_SalesLastYear',
    'CostYTD': 'CUSTOMER_SALESTERRITORY_CostYTD',
    'CostLastYear': 'CUSTOMER_SALESTERRITORY_CostLastYear'
}, inplace=True)

drop_modified_date_rowguid(customers)

customers.head()

,CUSTOMER_CUSTOMERS_CompanyName,CUSTOMER_CUSTOMERS_ContactName,CUSTOMER_CUSTOMERS_ContactTitle,CUSTOMER_CUSTOMERS_Address,CUSTOMER_CUSTOMERS_City,CUSTOMER_CUSTOMERS_Region,CUSTOMER_CUSTOMERS_PostalCode,CUSTOMER_CUSTOMERS_Country,CUSTOMER_CUSTOMERS_Phone,CUSTOMER_CUSTOMERS_Fax,CUSTOMER_CUSTOMER_Fname,CUSTOMER_CUSTOMER_Lname,CUSTOMER_CUSTOMER_State,CUSTOMER_CUSTOMERS_ID,CUSTOMER_CUSTOMER_PersonID,CUSTOMER_CUSTOMER_StoreID,CUSTOMER_CUSTOMER_TerritoryID,CUSTOMER_CUSTOMER_AccountNumber,CUSTOMER_STORE_Name,CUSTOMER_STORE_SalesPersonID,CUSTOMER_STORE_Demographics,CUSTOMER_SALESTERRITORY_Name,CUSTOMER_SALESTERRITORY_CountryRegionCode,CUSTOMER_SALESTERRITORY_Group,CUSTOMER_SALESTERRITORY_SalesYTD,CUSTOMER_SALESTERRITORY_SalesLastYear,CUSTOMER_SALESTERRITORY_CostYTD,CUSTOMER_SALESTERRITORY_CostLastYear
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,934.0,1.0,AW00000001,A Bike Store,280,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",Northwest,US,North America,7.887187e+06,3.298694e+06,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1028.0,1.0,AW00000002,Progressive Sports,283,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",Northwest,US,North America,7.887187e+06,3.298694e+06,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,642.0,4.0,AW00000003,Advanced Bike Components,277,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,932.0,4.0,AW00000004,Modular Cycle Systems,277,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,1026.0,4.0,AW00000005,Metropolitan Sports Supply,281,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0


### SpecialOffer

In [531]:
# merge specialoffer with specialofferproduct
specialoffers = pd.merge(adventureworks_sales_specialoffer, adventureworks_sales_specialofferproduct, on="SpecialOfferID")

# drop rowguid and modifieddate
drop_modified_date_rowguid(specialoffers)

# rename to correct columns like in database
specialoffers = specialoffers.rename(columns={
    'SpecialOfferID': 'SPECIALOFFER_SPECIALOFFER_ID',
    'ProductID': 'SPECIALOFFER_SPECIALOFFERPRODUCT_ProductID',
    'Description': 'SPECIALOFFER_SPECIALOFFER_Description',
    'DiscountPct': 'SPECIALOFFER_SPECIALOFFER_DiscountPCT',
    'Type': 'SPECIALOFFER_SPECIALOFFER_Type',
    'Category': 'SPECIALOFFER_SPECIALOFFER_Category',
    'StartDate': 'SPECIALOFFER_SPECIALOFFER_StartDate',
    'EndDate': 'SPECIALOFFER_SPECIALOFFER_EndDate',
    'MinQty': 'SPECIALOFFER_SPECIALOFFER_MinQty',
    'MaxQty': 'SPECIALOFFER_SPECIALOFFER_MaxQty'
})

specialoffers.head()

,SPECIALOFFER_SPECIALOFFER_ID,SPECIALOFFER_SPECIALOFFER_Description,SPECIALOFFER_SPECIALOFFER_DiscountPCT,SPECIALOFFER_SPECIALOFFER_Type,SPECIALOFFER_SPECIALOFFER_Category,SPECIALOFFER_SPECIALOFFER_StartDate,SPECIALOFFER_SPECIALOFFER_EndDate,SPECIALOFFER_SPECIALOFFER_MinQty,SPECIALOFFER_SPECIALOFFER_MaxQty,SPECIALOFFER_SPECIALOFFERPRODUCT_ProductID
0,1,No Discount,0.0,No Discount,No Discount,2011-05-01,2014-11-30,0,NaN,680
1,1,No Discount,0.0,No Discount,No Discount,2011-05-01,2014-11-30,0,NaN,706
2,1,No Discount,0.0,No Discount,No Discount,2011-05-01,2014-11-30,0,NaN,707
3,1,No Discount,0.0,No Discount,No Discount,2011-05-01,2014-11-30,0,NaN,708
4,1,No Discount,0.0,No Discount,No Discount,2011-05-01,2014-11-30,0,NaN,709


### CreditCard

In [532]:
# merge creditcard with personcreditcard
creditcards = pd.merge(adventureworks_sales_creditcard, adventureworks_sales_personcreditcard, on="CreditCardID")

# drop rowguid and modifieddate
drop_modified_date_rowguid(creditcards)

# rename to correct columns like in database
creditcards = creditcards.rename(columns={
    'CreditCardID': 'CREDITCARD_CREDITCARD_ID',
    'CardType': 'CREDITCARD_CREDITCARD_CardType',
    'CardNumber': 'CREDITCARD_CREDITCARD_CardNumber ',
    'ExpMonth': 'CREDITCARD_CREDITCARD_ExpMonth',
    'ExpYear': 'CREDITCARD_CREDITCARD_ExpYear',
    'BusinessEntityID': 'CREDITCARD_PERSONCREDITCARD_BusinessEntityID'
})

creditcards.head()

,CREDITCARD_CREDITCARD_ID,CREDITCARD_CREDITCARD_CardType,CREDITCARD_CREDITCARD_CardNumber,CREDITCARD_CREDITCARD_ExpMonth,CREDITCARD_CREDITCARD_ExpYear,CREDITCARD_PERSONCREDITCARD_BusinessEntityID
0,1,SuperiorCard,33332664695310,11,2006,4955
1,2,Distinguish,55552127249722,8,2005,13222
2,3,ColonialVoice,77778344838353,7,2005,7082
3,4,ColonialVoice,77774915718248,7,2006,9347
4,5,Vista,11114404600042,4,2005,11277


### Supplier

In [533]:
# rename
suppliers = northwind_suppliers.rename(columns={
    'SupplierID': 'SUPPLIER_SUPPLIERS_SupplierID',
    'CompanyName': 'SUPPLIER_SUPPLIERS_CompanyName',
    'ContactName': 'SUPPLIER_SUPPLIERS_ContactName',
    'ContactTitle': 'SUPPLIER_SUPPLIERS_ContactTitle',
    'Address': 'SUPPLIER_SUPPLIERS_Address',
    'City': 'SUPPLIER_SUPPLIERS_City',
    'Region': 'SUPPLIER_SUPPLIERS_Region',
    'PostalCode': 'SUPPLIER_SUPPLIERS_PostalCode',
    'Country': 'SUPPLIER_SUPPLIERS_Country',
    'Phone': 'SUPPLIER_SUPPLIERS_Phone',
    'Fax': 'SUPPLIER_SUPPLIERS_Fax',
    'HomePage': 'SUPPLIER_SUPPLIERS_HomePage'
})

suppliers.head()

,SUPPLIER_SUPPLIERS_SupplierID,SUPPLIER_SUPPLIERS_CompanyName,SUPPLIER_SUPPLIERS_ContactName,SUPPLIER_SUPPLIERS_ContactTitle,SUPPLIER_SUPPLIERS_Address,SUPPLIER_SUPPLIERS_City,SUPPLIER_SUPPLIERS_Region,SUPPLIER_SUPPLIERS_PostalCode,SUPPLIER_SUPPLIERS_Country,SUPPLIER_SUPPLIERS_Phone,SUPPLIER_SUPPLIERS_Fax,SUPPLIER_SUPPLIERS_HomePage
0,1,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,None,EC1 4SD,UK,(171) 555-2222,None,None
1,2,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,LA,70117,USA,(100) 555-4822,None,#CAJUN.HTM#
2,3,Grandma Kelly's Homestead,Regina Murphy,Sales Representative,707 Oxford Rd.,Ann Arbor,MI,48104,USA,(313) 555-5735,(313) 555-3349,None
3,4,Tokyo Traders,Yoshi Nagase,Marketing Manager,9-8 Sekimai Musashino-shi,Tokyo,None,100,Japan,(03) 3555-5011,None,None
4,5,Cooperativa de Quesos 'Las Cabras',Antonio del Valle Saavedra,Export Administrator,Calle del Rosal 4,Oviedo,Asturias,33007,Spain,(98) 598 76 54,None,None


### Currency

In [534]:
# combine currency and countryregioncurrency
currencies = pd.merge(adventureworks_sales_currency, adventureworks_sales_countryregioncurrency, on="CurrencyCode")

# now combine with currencyrate
currencies = pd.concat([currencies, adventureworks_sales_currencyrate], ignore_index=True)

#drop the modifieddate
drop_modified_date_rowguid(currencies)

#rename 
currencies = currencies.rename(columns={
    'CurrencyCode': 'CURRENCY_CURRENCY_CurrencyCode',
    'Name': 'CURRENCY_CURRENCY_Name',
    'CountryRegionCode': 'CURRENCY_COUNTRYREGIONCURRENCY_CountryRegionCode',
    'CurrencyRateID': 'CURRENCY_CURRENCYRATE_CurrencyRateID',
    'CurrencyRateDate': 'CURRENCY_CURRENCYRATE_CurrencyRateDate',
    'FromCurrencyCode': 'CURRENCY_CURRENCYRATE_FromCurrencyCode',
    'ToCurrencyCode': 'CURRENCY_CURRENCYRATE_ToCurrencyCode',
    'AverageRate': 'CURRENCY_CURRENCYRATE_AverageRate ',
    'EndOfDayRate': 'CURRENCY_CURRENCYRATE_EndOfDayRate',
})

currencies.head()

,CURRENCY_CURRENCY_CurrencyCode,CURRENCY_CURRENCY_Name,CURRENCY_COUNTRYREGIONCURRENCY_CountryRegionCode,CURRENCY_CURRENCYRATE_CurrencyRateID,CURRENCY_CURRENCYRATE_CurrencyRateDate,CURRENCY_CURRENCYRATE_FromCurrencyCode,CURRENCY_CURRENCYRATE_ToCurrencyCode,CURRENCY_CURRENCYRATE_AverageRate,CURRENCY_CURRENCYRATE_EndOfDayRate
0,AED,Emirati Dirham,AE,NaN,NaT,NaN,NaN,NaN,NaN
1,ARS,Argentine Peso,AR,NaN,NaT,NaN,NaN,NaN,NaN
2,ATS,Shilling,AT,NaN,NaT,NaN,NaN,NaN,NaN
3,AUD,Australian Dollar,AU,NaN,NaT,NaN,NaN,NaN,NaN
4,BBD,Barbados Dollar,BB,NaN,NaT,NaN,NaN,NaN,NaN


### Territory

In [535]:
# combine sales territory first
adventureworks_combined_territory = pd.merge(adventureworks_person_countryregion, adventureworks_person_stateprovince, on="CountryRegionCode", suffixes=('_pcr', '_sts'))

# now merge with salestaxrate
adventureworks_combined_territory = pd.merge(adventureworks_combined_territory, adventureworks_sales_salestaxrate, on="StateProvinceID", suffixes=('_st', '_sst'))

# combine northwind region with territory
nw_combined_territory = pd.merge(northwind_region, northwind_territories, on="RegionID")

# do the same for aenc
aenc_combined_regionstate = pd.merge(aenc_region, aenc_state, on="region")

#concatinate the tables together
territories = pd.concat([nw_combined_territory, aenc_combined_regionstate, adventureworks_combined_territory], ignore_index=True)

# drop unneeded rowguid and modifieddate
drop_modified_date_rowguid(territories)

# rename columns
territories = territories.rename(columns={
    'RegionID': 'TERRITORY_REGION_RegionID',
    'RegionDescription': 'TERRITORY_REGION_RegionDescription',
    'TerritoryID': 'TERRITORY_TERRITORIES_TerritoryID',
    'TerritoryDescription': 'TERRITORY_TERRITORIES_TerritoryDescription',
    'region': 'TERRITORY_REGION_Region',
    'state_id': 'TERRITORY_STATE_StateID',
    'state_name': 'TERRITORY_STATE_StateName',
    'state_capital': 'TERRITORY_STATE_StateCapital',
    'country': 'TERRITORY_STATE_Country',
    'CountryRegionCode': 'TERRITORY_COUNTRYREGION_CountryRegionCode',
    'Name_pcr': 'TERRITORY_COUNTRYREGION_Name',
    'StateProvinceID': 'TERRITORY_STATEPROVINCE_StateProvinceID',
    'StateProvinceCode': 'TERRITORY_STATEPROVINCE_StateProvinceCode',
    'IsOnlyStateProvinceFlag': 'TERRITORY_STATEPROVINCE_IsOnlyStateProvinceFlag',
    'Name_sts': 'TERRITORY_STATEPROVINCE_Name',
    'SalesTaxRateID': 'TERRITORY_SALESTAXRATE_SalesTaxRateID',
    'TaxType': 'TERRITORY_SALESTAXRATE_TaxType',
    'TaxRate': 'TERRITORY_SALESTAXRATE_TaxRate',
    'Name': 'TERRITORY_SALESTAXRATE_Name'
})

territories.head()

,TERRITORY_REGION_RegionID,TERRITORY_REGION_RegionDescription,TERRITORY_TERRITORIES_TerritoryID,TERRITORY_TERRITORIES_TerritoryDescription,TERRITORY_REGION_Region,TERRITORY_STATE_StateID,TERRITORY_STATE_StateName,TERRITORY_STATE_StateCapital,TERRITORY_STATE_Country,TERRITORY_COUNTRYREGION_CountryRegionCode,TERRITORY_COUNTRYREGION_Name,TERRITORY_STATEPROVINCE_StateProvinceID,TERRITORY_STATEPROVINCE_StateProvinceCode,TERRITORY_STATEPROVINCE_IsOnlyStateProvinceFlag,TERRITORY_STATEPROVINCE_Name,TERRITORY_SALESTAXRATE_SalesTaxRateID,TERRITORY_SALESTAXRATE_TaxType,TERRITORY_SALESTAXRATE_TaxRate,TERRITORY_SALESTAXRATE_Name
0,1.0,Eastern ...,01581,Westboro ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,Eastern ...,01730,Bedford ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,Eastern ...,01833,Georgetow ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,Eastern ...,02116,Boston ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,Eastern ...,02139,Cambridge ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### SalesOrderHeader

In [536]:
# combine orderdetail with orderheader
salesorderheaders = pd.merge(adventureworks_sales_salesorderdetail, adventureworks_sales_salesorderheader, on="SalesOrderID",
                        suffixes=('_sod', '_soh'))

# combine with salesorderheadersalesreason
salesorderheaders = pd.merge(salesorderheaders, adventureworks_sales_salesorderhearerrsaleseason, on="SalesOrderID")

# combine with salesreason
salesorderheaders = pd.merge(salesorderheaders, adventureworks_sales_salesreason, on="SalesReasonID")

#drop unneeded
drop_modified_date_rowguid(salesorderheaders)

# rename columns
salesorderheaders = salesorderheaders.rename(columns={
    'SalesOrderID': 'SALESORDERHEADER_SALESORDERDETAIL_SalesOrderID',
    'SalesOrderDetailID': 'SALESORDERHEADER_SALESORDERDETAIL_SalesOrderDetailID',
    'CarrierTrackingNumber': 'SALESORDERHEADER_SALESORDERDETAIL_CarrierTrackingNumber',
    'OrderQty': 'SALESORDERHEADER_SALESORDERDETAIL_OrderQty',
    'ProductID': 'SALESORDERHEADER_SALESORDERDETAIL_ProductID',
    'SpecialOfferID': 'SALESORDERHEADER_SALESORDERDETAIL_SpecialOfferID',
    'UnitPrice': 'SALESORDERHEADER_SALESORDERDETAIL_UnitPrice',
    'UnitPriceDiscount': 'SALESORDERHEADER_SALESORDERDETAIL_UnitPriceDiscount',
    'LineTotal': 'SALESORDERHEADER_SALESORDERDETAIL_LineTotal',
    'RevisionNumber': 'SALESORDERHEADER_SALESORDERHEADER_RevisionNumber',
    'OrderDate': 'SALESORDERHEADER_SALESORDERHEADER_OrderDate',
    'DueDate': 'SALESORDERHEADER_SALESORDERHEADER_DueDate',
    'ShipDate': 'SALESORDERHEADER_SALESORDERHEADER_ShipDate',
    'Status': 'SALESORDERHEADER_SALESORDERHEADER_Status',
    'OnlineOrderFlag': 'SALESORDERHEADER_SALESORDERHEADER_OnlineOrderFlag',
    'SalesOrderNumber': 'SALESORDERHEADER_SALESORDERHEADER_SalesOrderNumber',
    'PurchaseOrderNumber': 'SALESORDERHEADER_SALESORDERHEADER_PurchaseOrderNumber',
    'AccountNumber': 'SALESORDERHEADER_SALESORDERHEADER_AccountNumber',
    'CustomerID': 'SALESORDERHEADER_SALESORDERHEADER_CustomerID',
    'SalesPersonID': 'SALESORDERHEADER_SALESORDERHEADER_SalesPersonID',
    'TerritoryID': 'SALESORDERHEADER_SALESORDERHEADER_TerritoryID',
    'BillToAddressID': 'SALESORDERHEADER_SALESORDERHEADER_BillToAddress',
    'ShipToAddressID': 'SALESORDERHEADER_SALESORDERHEADER_ShipToAddress',
    'ShipMethodID': 'SALESORDERHEADER_SALESORDERHEADER_ShipMethodID',
    'CreditCardID': 'SALESORDERHEADER_SALESORDERHEADER_CreditCardID',
    'CreditCardApprovalCode': 'SALESORDERHEADER_SALESORDERHEADER_CreditCardApprovalCode',
    'CurrencyRateID': 'SALESORDERHEADER_SALESORDERHEADER_CurrencyRateID',
    'SubTotal': 'SALESORDERHEADER_SALESORDERHEADER_SubTotal',
    'TaxAmt': 'SALESORDERHEADER_SALESORDERHEADER_TaxAmt',
    'Freight': 'SALESORDERHEADER_SALESORDERHEADER_Freight',
    'TotalDue': 'SALESORDERHEADER_SALESORDERHEADER_TotalDue',
    'Comment': 'SALESORDERHEADER_SALESORDERHEADER_Comment',
    'SalesReasonID': 'SALESORDERHEADER_SALESREASON_SalesReasonID',
    'Name': 'SALESORDERHEADER_SALESREASON_Name',
    'ReasonType': 'SALESORDERHEADER_SALESREASON_ReasonType',
})

salesorderheaders.head()


,SALESORDERHEADER_SALESORDERDETAIL_SalesOrderID,SALESORDERHEADER_SALESORDERDETAIL_SalesOrderDetailID,SALESORDERHEADER_SALESORDERDETAIL_CarrierTrackingNumber,SALESORDERHEADER_SALESORDERDETAIL_OrderQty,SALESORDERHEADER_SALESORDERDETAIL_ProductID,SALESORDERHEADER_SALESORDERDETAIL_SpecialOfferID,SALESORDERHEADER_SALESORDERDETAIL_UnitPrice,SALESORDERHEADER_SALESORDERDETAIL_UnitPriceDiscount,SALESORDERHEADER_SALESORDERDETAIL_LineTotal,SALESORDERHEADER_SALESORDERHEADER_RevisionNumber,SALESORDERHEADER_SALESORDERHEADER_OrderDate,SALESORDERHEADER_SALESORDERHEADER_DueDate,SALESORDERHEADER_SALESORDERHEADER_ShipDate,SALESORDERHEADER_SALESORDERHEADER_Status,SALESORDERHEADER_SALESORDERHEADER_OnlineOrderFlag,SALESORDERHEADER_SALESORDERHEADER_SalesOrderNumber,SALESORDERHEADER_SALESORDERHEADER_PurchaseOrderNumber,SALESORDERHEADER_SALESORDERHEADER_AccountNumber,SALESORDERHEADER_SALESORDERHEADER_CustomerID,SALESORDERHEADER_SALESORDERHEADER_SalesPersonID,SALESORDERHEADER_SALESORDERHEADER_TerritoryID,SALESORDERHEADER_SALESORDERHEADER_BillToAddress,SALESORDERHEADER_SALESORDERHEADER_ShipToAddress,SALESORDERHEADER_SALESORDERHEADER_ShipMethodID,SALESORDERHEADER_SALESORDERHEADER_CreditCardID,SALESORDERHEADER_SALESORDERHEADER_CreditCardApprovalCode,SALESORDERHEADER_SALESORDERHEADER_CurrencyRateID,SALESORDERHEADER_SALESORDERHEADER_SubTotal,SALESORDERHEADER_SALESORDERHEADER_TaxAmt,SALESORDERHEADER_SALESORDERHEADER_Freight,SALESORDERHEADER_SALESORDERHEADER_TotalDue,SALESORDERHEADER_SALESORDERHEADER_Comment,SALESORDERHEADER_SALESREASON_SalesReasonID,SALESORDERHEADER_SALESREASON_Name,SALESORDERHEADER_SALESREASON_ReasonType
0,43697,353,None,1,749,1,3578.27,0.0,3578.27,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43697,None,10-4030-021768,21768,NaN,6,23148,23148,1,4319.0,530200Vi22686,4.0,3578.27,286.2616,89.4568,3953.9884,None,5,Manufacturer,Other
1,43697,353,None,1,749,1,3578.27,0.0,3578.27,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43697,None,10-4030-021768,21768,NaN,6,23148,23148,1,4319.0,530200Vi22686,4.0,3578.27,286.2616,89.4568,3953.9884,None,9,Quality,Other
2,43702,358,None,1,750,1,3578.27,0.0,3578.27,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43702,None,10-4030-027645,27645,NaN,4,22584,22584,1,8087.0,1230194Vi41919,NaN,3578.27,286.2616,89.4568,3953.9884,None,5,Manufacturer,Other
3,43702,358,None,1,750,1,3578.27,0.0,3578.27,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43702,None,10-4030-027645,27645,NaN,4,22584,22584,1,8087.0,1230194Vi41919,NaN,3578.27,286.2616,89.4568,3953.9884,None,9,Quality,Other
4,43703,359,None,1,749,1,3578.27,0.0,3578.27,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43703,None,10-4030-016624,16624,NaN,9,23967,23967,1,9497.0,1234632Vi48985,15.0,3578.27,286.2616,89.4568,3953.9884,None,5,Manufacturer,Other


### PurchaseOrderHeader

In [537]:
# combine purchaseorderdetail with purchaseorderheader
purchaseorderheaders = pd.merge(adventureworks_purchasing_purchaseorderheader, adventureworks_purchasing_purchaseorderdetail, on="PurchaseOrderID")

# combine with purchase shipmethod
purchaseorderheaders = pd.merge(purchaseorderheaders, adventureworks_purchasing_shipmethod, on="ShipMethodID")

#combine wiith vendor
purchaseorderheaders = pd.merge(purchaseorderheaders, adventureworks_purchasing_vendor, left_on='VendorID', right_on='BusinessEntityID',
                        suffixes=('_poh', '_pov'))

#drop unneeded
drop_modified_date_rowguid(purchaseorderheaders)
purchaseorderheaders.drop(columns=['BusinessEntityID'], inplace=True)

# rename columns
purchaseorderheaders = purchaseorderheaders.rename(columns={
    'PurchaseOrderID': 'PURCHASEORDERHEADER_PURCHASEORDERHEADER_PurchaseOrderID',
    'RevisionNumber': 'PURCHASEORDERHEADER_PURCHASEORDERHEADER_RevisionNumber',
    'Status': 'PURCHASEORDERHEADER_PURCHASEORDERHEADER_Status',
    'EmployeeID': 'PURCHASEORDERHEADER_PURCHASEORDERHEADER_EmployeeID',
    'VendorID': 'PURCHASEORDERHEADER_PURCHASEORDERHEADER_VendorID',
    'ShipMethodID': 'PURCHASEORDERHEADER_PURCHASEORDERHEADER_ShipMethodID',
    'OrderDate': 'PURCHASEORDERHEADER_PURCHASEORDERHEADER_OrderDate',
    'ShipDate': 'PURCHASEORDERHEADER_PURCHASEORDERHEADER_ShipDate',
    'SubTotal': 'PURCHASEORDERHEADER_PURCHASEORDERHEADER_SubTotal',
    'TaxAmt': 'PURCHASEORDERHEADER_PURCHASEORDERHEADER_TaxAmt',
    'Freight': 'PURCHASEORDERHEADER_PURCHASEORDERHEADER_Freight',
    'TotalDue': 'PURCHASEORDERHEADER_PURCHASEORDERHEADER_TotalDue',
    'PurchaseOrderDetailID': 'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_PurchaseOrderDetailID',
    'DueDate': 'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_DueDate',
    'OrderQty': 'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_OrderQty',
    'ProductID': 'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_ProductID',
    'UnitPrice': 'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_UnitPrice',
    'LineTotal': 'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_LineTotal',
    'ReceivedQty': 'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_ReceivedQty',
    'RejectedQty': 'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_RejectedQty',
    'StockedQty': 'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_StockedQty',
    'Name_poh': 'PURCHASEORDERHEADER_SHIPMETHOD_Name',
    'ShipBase': 'PURCHASEORDERHEADER_SHIPMETHOD_ShipBase',
    'ShipRate': 'PURCHASEORDERHEADER_SHIPMETHOD_ShipRate',
    'AccountNumber': 'PURCHASEORDERHEADER_VENDOR_AccountNumber',
    'Name_pov': 'PURCHASEORDERHEADER_VENDOR_Name',
    'CreditRating': 'PURCHASEORDERHEADER_VENDOR_CreditRating',
    'PreferredVendorStatus': 'PURCHASEORDERHEADER_VENDOR_PreferredVendorStatus',
    'ActiveFlag': 'PURCHASEORDERHEADER_VENDOR_ActiveFlag',
    'PurchasingWebServiceURL': 'PURCHASEORDERHEADER_VENDOR_PurchasingWebServiceURL'
})

purchaseorderheaders.head()


,PURCHASEORDERHEADER_PURCHASEORDERHEADER_PurchaseOrderID,PURCHASEORDERHEADER_PURCHASEORDERHEADER_RevisionNumber,PURCHASEORDERHEADER_PURCHASEORDERHEADER_Status,PURCHASEORDERHEADER_PURCHASEORDERHEADER_EmployeeID,PURCHASEORDERHEADER_PURCHASEORDERHEADER_VendorID,PURCHASEORDERHEADER_PURCHASEORDERHEADER_ShipMethodID,PURCHASEORDERHEADER_PURCHASEORDERHEADER_OrderDate,PURCHASEORDERHEADER_PURCHASEORDERHEADER_ShipDate,PURCHASEORDERHEADER_PURCHASEORDERHEADER_SubTotal,PURCHASEORDERHEADER_PURCHASEORDERHEADER_TaxAmt,PURCHASEORDERHEADER_PURCHASEORDERHEADER_Freight,PURCHASEORDERHEADER_PURCHASEORDERHEADER_TotalDue,PURCHASEORDERHEADER_PURCHASEORDERDETAIL_PurchaseOrderDetailID,PURCHASEORDERHEADER_PURCHASEORDERDETAIL_DueDate,PURCHASEORDERHEADER_PURCHASEORDERDETAIL_OrderQty,PURCHASEORDERHEADER_PURCHASEORDERDETAIL_ProductID,PURCHASEORDERHEADER_PURCHASEORDERDETAIL_UnitPrice,PURCHASEORDERHEADER_PURCHASEORDERDETAIL_LineTotal,PURCHASEORDERHEADER_PURCHASEORDERDETAIL_ReceivedQty,PURCHASEORDERHEADER_PURCHASEORDERDETAIL_RejectedQty,PURCHASEORDERHEADER_PURCHASEORDERDETAIL_StockedQty,PURCHASEORDERHEADER_SHIPMETHOD_Name,PURCHASEORDERHEADER_SHIPMETHOD_ShipBase,PURCHASEORDERHEADER_SHIPMETHOD_ShipRate,PURCHASEORDERHEADER_VENDOR_AccountNumber,PURCHASEORDERHEADER_VENDOR_Name,PURCHASEORDERHEADER_VENDOR_CreditRating,PURCHASEORDERHEADER_VENDOR_PreferredVendorStatus,PURCHASEORDERHEADER_VENDOR_ActiveFlag,PURCHASEORDERHEADER_VENDOR_PurchasingWebServiceURL
0,1,4,4,258,1580,3,2011-04-16,2011-04-25,201.0400,16.0832,5.0260,222.1492,1,2011-04-30,4,1,50.2600,201.0400,3.0,0.0,3.0,OVERSEAS - DELUXE,29.95,2.99,LITWARE0001,"Litware, Inc.",1,True,True,www.litwareinc.com/
1,2,4,1,254,1496,5,2011-04-16,2011-04-25,272.1015,21.7681,6.8025,300.6721,2,2011-04-30,3,359,45.1200,135.3600,3.0,0.0,3.0,CARGO TRANSPORT 5,8.99,1.49,ADVANCED0001,Advanced Bicycles,1,True,True,None
2,2,4,1,254,1496,5,2011-04-16,2011-04-25,272.1015,21.7681,6.8025,300.6721,3,2011-04-30,3,360,45.5805,136.7415,3.0,0.0,3.0,CARGO TRANSPORT 5,8.99,1.49,ADVANCED0001,Advanced Bicycles,1,True,True,None
3,3,4,4,257,1494,2,2011-04-16,2011-04-25,8847.3000,707.7840,221.1825,9776.2665,4,2011-04-30,550,530,16.0860,8847.3000,550.0,0.0,550.0,ZY - EXPRESS,9.95,1.99,ALLENSON0001,Allenson Cycles,2,True,True,None
4,4,4,3,261,1650,5,2011-04-16,2011-04-25,171.0765,13.6861,4.2769,189.0395,5,2011-04-30,3,4,57.0255,171.0765,2.0,1.0,1.0,CARGO TRANSPORT 5,8.99,1.49,AMERICAN0001,American Bicycles and Wheels,1,True,True,None


### Date

In [538]:
def calculate_periods(periods_in_years): 
    return 365 * periods_in_years + (periods_in_years // 4) # divide by 4 for leap years

periods = calculate_periods(50)

date_table = {
    'DATE_ID': pd.date_range(start='1996-01-01', periods=periods).strftime('%Y%m%d').astype(int),
    'DATE_Date': pd.date_range(start='1996-01-01', periods=periods),
    'DATE_Weekday': pd.date_range(start='1996-01-01', periods=periods).strftime('%A'),
    'DATE_WeekdayNum': pd.date_range(start='1996-01-01', periods=periods).weekday + 1,
    'DATE_DayMonth': pd.date_range(start='1996-01-01', periods=periods).day,
    'DATE_DayOfYear': pd.date_range(start='1996-01-01', periods=periods).dayofyear,
    'DATE_WeekOfYear': pd.date_range(start='1996-01-01', periods=periods).isocalendar().week,
    'DATE_MonthNum': pd.date_range(start='1996-01-01', periods=periods).month,
    'DATE_MonthName': pd.date_range(start='1996-01-01', periods=periods).strftime('%B'),
    'DATE_MonthNameShort': pd.date_range(start='1996-01-01', periods=periods).strftime('%b'),
    'DATE_Quarter': pd.date_range(start='1996-01-01', periods=periods).quarter,
    'DATE_Year': pd.date_range(start='1996-01-01', periods=periods).year,
    'DATE_FirstDayOfMonth': pd.date_range(start='1996-01-01', periods=periods).to_period('M').start_time,
    'DATE_LastDayOfMonth': pd.date_range(start='1996-01-01', periods=periods).to_period('M').end_time.date,
    'DATE_YYYYMM': pd.date_range(start='1996-01-01', periods=periods).strftime('%Y-%m'),
    'DATE_WeekendIndr': pd.date_range(start='1996-01-01', periods=periods).weekday // 5
}

date_table = pd.DataFrame(date_table)
date_table['DATE_WeekendIndr'] = date_table['DATE_WeekendIndr'].replace({0: 'weekday', 1: 'weekend'})

date_table.tail()

,DATE_ID,DATE_Date,DATE_Weekday,DATE_WeekdayNum,DATE_DayMonth,DATE_DayOfYear,DATE_WeekOfYear,DATE_MonthNum,DATE_MonthName,DATE_MonthNameShort,DATE_Quarter,DATE_Year,DATE_FirstDayOfMonth,DATE_LastDayOfMonth,DATE_YYYYMM,DATE_WeekendIndr
2045-12-26,20451226,2045-12-26,Tuesday,2,26,360,52,12,December,Dec,4,2045,2045-12-01,2045-12-31,2045-12,weekday
2045-12-27,20451227,2045-12-27,Wednesday,3,27,361,52,12,December,Dec,4,2045,2045-12-01,2045-12-31,2045-12,weekday
2045-12-28,20451228,2045-12-28,Thursday,4,28,362,52,12,December,Dec,4,2045,2045-12-01,2045-12-31,2045-12,weekday
2045-12-29,20451229,2045-12-29,Friday,5,29,363,52,12,December,Dec,4,2045,2045-12-01,2045-12-31,2045-12,weekday
2045-12-30,20451230,2045-12-30,Saturday,6,30,364,52,12,December,Dec,4,2045,2045-12-01,2045-12-31,2045-12,weekend


### Time

In [539]:
# Define the number of minutes in a day
minutes_in_a_day = 24 * 60

# Create the data dictionary for time dimension
time_table = {
    'TIME_ID': list(range(minutes_in_a_day)),
    'TIME_Hour': [i // 60 for i in range(minutes_in_a_day)],
    'TIME_Minute': [i % 60 for i in range(minutes_in_a_day)],
    'TIME_HourMinute': [f'{hour:02d}:{minute:02d}' for hour in range(24) for minute in range(60)],
}

# Convert the dictionary to a DataFrame
time_table = pd.DataFrame(time_table)

time_table.head()

,TIME_ID,TIME_Hour,TIME_Minute,TIME_HourMinute
0,0,0,0,00:00
1,1,0,1,00:01
2,2,0,2,00:02
3,3,0,3,00:03
4,4,0,4,00:04


### TransactionHistory

In [540]:
transactionhistories = adventureworks_production_transactionhistory

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(transactionhistories)

# renaming the columns
transactionhistories.rename(columns={'TransactionID': 'TRANSACTIONHISTORY_TRANSACTIONHISTORY_TransactionID', 'ProductID': 'TRANSACTIONHISTORY_TRANSACTIONHISTORY_ProductID', 'ReferenceOrderID': 'TRANSACTIONHISTORY_TRANSACTIONHISTORY_ReferenceOrderID', 'ReferenceOrderLineID': 'TRANSACTIONHISTORY_TRANSACTIONHISTORY_ReferenceOrderLineID', 'TransactionDate': 'TRANSACTIONHISTORY_TRANSACTIONHISTORY_TransactionDate', 'TransactionType': 'TRANSACTIONHISTORY_TRANSACTIONHISTORY_TransactionType', 'Quantity': 'TRANSACTIONHISTORY_TRANSACTIONHISTORY_Quantity', 'ActualCost': 'TRANSACTIONHISTORY_TRANSACTIONHISTORY_ActualCost'}, inplace=True)

transactionhistories.head()

,TRANSACTIONHISTORY_TRANSACTIONHISTORY_TransactionID,TRANSACTIONHISTORY_TRANSACTIONHISTORY_ProductID,TRANSACTIONHISTORY_TRANSACTIONHISTORY_ReferenceOrderID,TRANSACTIONHISTORY_TRANSACTIONHISTORY_ReferenceOrderLineID,TRANSACTIONHISTORY_TRANSACTIONHISTORY_TransactionDate,TRANSACTIONHISTORY_TRANSACTIONHISTORY_TransactionType,TRANSACTIONHISTORY_TRANSACTIONHISTORY_Quantity,TRANSACTIONHISTORY_TRANSACTIONHISTORY_ActualCost
0,100000,784,41590,0,2013-07-31,W,2,0.0
1,100001,794,41591,0,2013-07-31,W,1,0.0
2,100002,797,41592,0,2013-07-31,W,1,0.0
3,100003,798,41593,0,2013-07-31,W,1,0.0
4,100004,799,41594,0,2013-07-31,W,1,0.0


### TransactionHistoryArchive

In [541]:
transactionhistoryarchives = adventureworks_production_transactionhistoryarchive

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(transactionhistoryarchives)

# renaming the columns
transactionhistoryarchives.rename(columns={'TransactionID': 'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_TransactionID', 'ProductID': 'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_ProductID', 'ReferenceOrderID': 'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_ReferenceOrderID', 'ReferenceOrderLineID': 'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_ReferenceOrderLineID', 'TransactionDate': 'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_TransactionDate', 'TransactionType': 'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_TransactionType', 'Quantity': 'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_Quantity', 'ActualCost': 'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_ActualCost'}, inplace=True)

transactionhistoryarchives.head()

,TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_TransactionID,TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_ProductID,TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_ReferenceOrderID,TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_ReferenceOrderLineID,TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_TransactionDate,TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_TransactionType,TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_Quantity,TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_ActualCost
0,1,1,1,1,2011-04-16,P,4,50.2600
1,2,359,2,1,2011-04-16,P,3,45.1200
2,3,360,2,2,2011-04-16,P,3,45.5805
3,4,530,3,1,2011-04-16,P,550,16.0860
4,5,4,4,1,2011-04-16,P,3,57.0255


### ProductReview

In [542]:
productreviews = adventureworks_production_productreview

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(productreviews)

# renaming the columns
productreviews.rename(columns={'ProductReviewID': 'PRODUCTREVIEW_PRODUCTREVIEW_ProductReviewID', 'ProductID': 'PRODUCTREVIEW_PRODUCTREVIEW_ProductID', 'ReviewerName': 'PRODUCTREVIEW_PRODUCTREVIEW_ReviewerName', 'ReviewDate': 'PRODUCTREVIEW_PRODUCTREVIEW_ReviewDate', 'EmailAddress': 'PRODUCTREVIEW_PRODUCTREVIEW_EmailAddress', 'Rating': 'PRODUCTREVIEW_PRODUCTREVIEW_Rating', 'Comments': 'PRODUCTREVIEW_PRODUCTREVIEW_Comments'}, inplace=True)

productreviews.head()

,PRODUCTREVIEW_PRODUCTREVIEW_ProductReviewID,PRODUCTREVIEW_PRODUCTREVIEW_ProductID,PRODUCTREVIEW_PRODUCTREVIEW_ReviewerName,PRODUCTREVIEW_PRODUCTREVIEW_ReviewDate,PRODUCTREVIEW_PRODUCTREVIEW_EmailAddress,PRODUCTREVIEW_PRODUCTREVIEW_Rating,PRODUCTREVIEW_PRODUCTREVIEW_Comments
0,1,709,John Smith,2013-09-18,john@fourthcoffee.com,5,I can't believe I'm singing the praises of a p...
1,2,937,David,2013-11-13,david@graphicdesigninstitute.com,4,"A little on the heavy side, but overall the en..."
2,3,937,Jill,2013-11-15,jill@margiestravel.com,2,Maybe it's just because I'm new to mountain bi...
3,4,798,Laura Norman,2013-11-15,laura@treyresearch.net,5,The Road-550-W from Adventure Works Cycles is ...


### Inventory

In [543]:
inventory_aw = adventureworks_production_productinventory.rename(columns={
    'ProductID' : 'INVENTORY_PRODUCT_ID',
    'LocationID' : 'INVENTORY_LOCATION_ID',
    'Shelf' : 'INVENTORY_SHELF',
    'Bin' : 'INVENTORY_Bin',
    'Quantity' : 'INVENTORY_Quantity'
})

inventory_aw.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)

inventory_aenc = aenc_product[['id', 'quantity']].rename(columns={'id' : 'INVENTORY_PRODUCT_ID', 'quantity' : 'INVENTORY_Quantity'})
inventory_northwind = northwind_products[['ProductID', 'UnitsInStock']].rename(columns={'ProductID' : 'INVENTORY_PRODUCT_ID', 'UnitsInStock' : 'INVENTORY_Quantity'})

inventory_combined = pd.concat([inventory_aw, inventory_aenc, inventory_northwind], ignore_index=True)
inventory_combined

,INVENTORY_PRODUCT_ID,INVENTORY_LOCATION_ID,INVENTORY_SHELF,INVENTORY_Bin,INVENTORY_Quantity
0,1,1.0,A,1.0,408
1,1,6.0,B,5.0,324
2,1,50.0,A,5.0,353
3,2,1.0,A,2.0,427
4,2,6.0,B,1.0,318
...,...,...,...,...,...
1141,73,NaN,NaN,NaN,101
1142,74,NaN,NaN,NaN,4
1143,75,NaN,NaN,NaN,125
1144,76,NaN,NaN,NaN,57


### Products

In [544]:
product_aenc = aenc_product.rename(columns={'id' : 'PRODUCT_PRODUCT_ID',
                                            'name' : 'PRODUCT_PRODUCT_Name',
                                            'description' : 'PRODUCT_PRODUCTDESCRIPTION_Desc',
                                            'prod_size' : 'PRODUCT_PRODUCT_Size',
                                            'color' : 'PRODUCT_PRODUCT_Color',
                                            'unit_price' : 'PRODUCT_PRODUCT_UnitPrice',
                                            'picture_name' : 'PRODUCT_PRODUCTPHOTO_ThumbnailPhotoFileName',
                                            'Category' : 'PRODUCT_PRODUCTCATEGORY_Category',
                                            })

product_aenc.drop(columns=['quantity'], inplace=True)

In [545]:
product_northwind = northwind_products.merge(northwind_categories, on='CategoryID', how='left')

product_northwind = product_northwind.rename(columns={'ProductID' : 'PRODUCT_PRODUCT_ID',
                                                        'ProductName' : 'PRODUCT_PRODUCT_Name',
                                                        'SupplierID' : 'PRODUCT_PRODUCT_SupplierID',
                                                        'CategoryID' : 'PRODUCT_PRODUCTSUBCATEGORY_CategoryID',
                                                        'QuantityPerUnit' : 'PRODUCT_PRODUCT_QuantityPerUnit',
                                                        'UnitPrice' : 'PRODUCT_PRODUCT_UnitPrice',
                                                        'UnitsOnOrder' : 'PRODUCT_PRODUCT_UnitsOnOrder',
                                                        'ReorderLevel' : 'PRODUCT_PRODUCT_ReorderLevel',
                                                        'Discontinued' : 'PRODUCT_PRODUCT_Discontinued',
                                                        'CategoryName' : 'PRODUCT_PRODUCTCATEGORY_Category',
                                                        'Description' : 'PRODUCT_PRODUCTDESCRIPTION_Desc',
                                                        'Picture' : 'PRODUCT_CATEGORY_Picture'
                                                        })

product_northwind.drop(columns=['UnitsInStock'], inplace=True)

In [546]:
# combining all adventureworks product data
product_adventure_works = pd.merge(adventureworks_production_product, adventureworks_production_unitmeasure, left_on='SizeUnitMeasureCode', right_on='UnitMeasureCode', how='left')
product_adventure_works.rename(columns={'Name_y' : 'PRODUCT_PRODUCT_SizeUnitMeasureName'}, inplace=True)
product_adventure_works.rename(columns={'SizeUnitMeasureCode' : 'PRODUCT_PRODUCT_SizeUnitMeasureCode'}, inplace=True)
product_adventure_works.drop(columns=['UnitMeasureCode', 'rowguid', 'ModifiedDate_x', 'ModifiedDate_y'], inplace=True)

product_adventure_works = pd.merge(product_adventure_works, adventureworks_production_unitmeasure, left_on='WeightUnitMeasureCode', right_on='UnitMeasureCode', how='left')
product_adventure_works.rename(columns={'Name' : 'PRODUCT_PRODUCT_WeightUnitMeasureName'}, inplace=True)
product_adventure_works.rename(columns={'WeightUnitMeasureCode' : 'PRODUCT_PRODUCT_WeightUnitMeasureCode'}, inplace=True)
product_adventure_works.drop(columns=['UnitMeasureCode', 'ModifiedDate'], inplace=True)

product_adventure_works = pd.merge(product_adventure_works, adventureworks_production_productsubcategory, left_on='ProductSubcategoryID', right_on='ProductSubcategoryID', how='left')
product_adventure_works.rename(columns={'Name' : 'PRODUCT_PRODUCTSUBCATEGORY_SubCategory'}, inplace=True)
product_adventure_works.rename(columns={'ProductSubcategoryID' : 'PRODUCT_PRODUCT_SubCategoryID'}, inplace=True)
product_adventure_works.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)

product_adventure_works = pd.merge(product_adventure_works, adventureworks_production_productcategory, left_on='ProductCategoryID', right_on='ProductCategoryID', how='left')
product_adventure_works.rename(columns={'Name' : 'PRODUCT_PRODUCTCATEGORY_Category'}, inplace=True)
product_adventure_works.rename(columns={'ProductCategoryID' : 'PRODUCT_PRODUCTSUBCATEGORY_CategoryID'}, inplace=True)
product_adventure_works.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)

product_adventure_works = pd.merge(product_adventure_works, adventureworks_production_productmodel, left_on='ProductModelID', right_on='ProductModelID', how='left')
product_adventure_works.rename(columns={'Name' : 'PRODUCT_PRODUCTMODEL_Name'}, inplace=True)
product_adventure_works.rename(columns={'CatalogDescription' : 'PRODUCT_PRODUCTMODEL_CatalogDescription'}, inplace=True)
product_adventure_works.rename(columns={'Instructions' : 'PRODUCT_PRODUCTMODEL_Instructions'}, inplace=True)
product_adventure_works.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)

product_adventure_works = pd.merge(product_adventure_works, adventureworks_production_productmodelillustration, left_on='ProductModelID', right_on='ProductModelID', how='left')
product_adventure_works.rename(columns={'ProductModelID' : 'PRODUCT_PRODUCT_ModelID'}, inplace=True)
product_adventure_works.drop(columns=['ModifiedDate'], inplace=True)

product_adventure_works = pd.merge(product_adventure_works, adventureworks_production_illustration, left_on='IllustrationID', right_on='IllustrationID', how='left')
product_adventure_works.rename(columns={'IllustrationID' : 'PRODUCT_PRODUCTMODELILLUSTRATION_IllustrationID'}, inplace=True)
product_adventure_works.rename(columns={'Diagram' : 'PRODUCT_ILLUSTRATION_Diagram'}, inplace=True)
product_adventure_works.drop(columns=['ModifiedDate'], inplace=True)

product_adventure_works = pd.merge(product_adventure_works, adventureworks_production_productmodelproductdescriptionculture , left_on='PRODUCT_PRODUCT_ModelID', right_on='ProductModelID', how='left')
product_adventure_works.drop(columns=['ProductModelID'], inplace=True)
product_adventure_works.drop(columns=['ModifiedDate'], inplace=True)

product_adventure_works = pd.merge(product_adventure_works, adventureworks_production_culture , left_on='CultureID', right_on='CultureID', how='left')
product_adventure_works.rename(columns={'CultureID' : 'PRODUCT_PMPDC_CultureID'}, inplace=True)
product_adventure_works.rename(columns={'Name' : 'PRODUCT_CULTURE_Name'}, inplace=True)
product_adventure_works.drop(columns=['ModifiedDate'], inplace=True)

product_adventure_works = pd.merge(product_adventure_works, adventureworks_production_productdescription, left_on='ProductDescriptionID', right_on='ProductDescriptionID', how='left')
product_adventure_works.rename(columns={'ProductDescriptionID' : 'PRODUCT_PMPDC_DescriptionID'}, inplace=True)
product_adventure_works.rename(columns={'Description' : 'PRODUCT_PRODUCTDESCRIPTION_Desc'}, inplace=True)
product_adventure_works.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)

product_adventure_works = pd.merge(product_adventure_works, adventureworks_production_productproductphoto, left_on='ProductID', right_on='ProductID', how='left')
product_adventure_works.rename(columns={'Primary' : 'PRODUCT_PRODUCTPRODUCTPHOTO_Primary'}, inplace=True)
product_adventure_works.drop(columns=['ModifiedDate'], inplace=True)

product_adventure_works = pd.merge(product_adventure_works, adventureworks_production_productphoto, left_on='ProductPhotoID', right_on='ProductPhotoID', how='left')
product_adventure_works.rename(columns={'ProductPhotoID' : 'PRODUCT_PRODUCTPRODUCTPHOTO_PhotoID'}, inplace=True)
product_adventure_works.rename(columns={'ThumbNailPhotoHexString' : 'PRODUCT_PRODUCTPHOTO_ThumbnailPhotoHexString'}, inplace=True)
product_adventure_works.rename(columns={'LargePhotoHexString' : 'PRODUCT_PRODUCTPHOTO_LargePhotoHexString'}, inplace=True)
product_adventure_works.rename(columns={'ThumbNailPhotoFileName' : 'PRODUCT_PRODUCTPHOTO_ThumbnailPhotoFileName'}, inplace=True)
product_adventure_works.rename(columns={'LargePhotoFileName' : 'PRODUCT_PRODUCTPHOTO_LargePhotoFileName'}, inplace=True)
product_adventure_works.drop(columns=['ModifiedDate'], inplace=True)

product_adventure_works = product_adventure_works.rename(columns={'ProductID' : 'PRODUCT_PRODUCT_ID',
                                                                    'Name_x' : 'PRODUCT_PRODUCT_Name',
                                                                    'ProductNumber' : 'PRODUCT_PRODUCT_Number',
                                                                    'MakeFlag' : 'PRODUCT_PRODUCT_MakeFlag',
                                                                    'FinishedGoodsFlag' : 'PRODUCT_PRODUCT_FinishedGoodsFlag',
                                                                    'SafetyStockLevel' : 'PRODUCT_PRODUCT_SafetyStockLevel',
                                                                    'ReorderPoint' : 'PRODUCT_PRODUCT_ReorderPoint',
                                                                    'StandardCost' : 'PRODUCT_PRODUCT_StandardCost',
                                                                    'ListPrice' : 'PRODUCT_PRODUCT_ListPrice',
                                                                    'Size' : 'PRODUCT_PRODUCT_Size',
                                                                    'Weight' : 'PRODUCT_PRODUCT_Weight',
                                                                    'DaysToManufacture' : 'PRODUCT_PRODUCT_DaysToManufacture',
                                                                    'ProductLine' : 'PRODUCT_PRODUCT_ProductLine',
                                                                    'Class' : 'PRODUCT_PRODUCT_Class',
                                                                    'Style' : 'PRODUCT_PRODUCT_Style',
                                                                    'SellStartDate' : 'PRODUCT_PRODUCT_SellStartDate',
                                                                    'SellEndDate' : 'PRODUCT_PRODUCT_SellEndDate',
                                                                    'DiscontinuedDate' : 'PRODUCT_PRODUCT_DiscountedDate',
                                                                    'Color' : 'PRODUCT_PRODUCT_Color',
                                                                    }) 

# # applying the data types to the columns
product_adventure_works['PRODUCT_PRODUCT_ID'] = product_adventure_works['PRODUCT_PRODUCT_ID'].astype(float) # casting to float seems counterintuitive but it is necessary to avoid an error in Pandas

product_combined = pd.concat([product_adventure_works, product_aenc, product_northwind], ignore_index=True)

C:\Users\yunyi\AppData\Local\Temp\ipykernel_43504\290765978.py:87: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  product_combined = pd.concat([product_adventure_works, product_aenc, product_northwind], ignore_index=True)


## Loading the data into the UnitedOutdoors datawarehouse

### Date

In [547]:
date_dtypes = {
    'DATE_ID': Integer,
    'DATE_Date': DATE,
    'DATE_Weekday': VARCHAR(10),
    'DATE_WeekdayNum': Integer,
    'DATE_DayMonth': Integer,
    'DATE_DayOfYear': Integer,
    'DATE_WeekOfYear': Integer,
    'DATE_MonthNum': Integer,
    'DATE_MonthName': VARCHAR(10),
    'DATE_MonthNameShort': CHAR(10),
    'DATE_Quarter': Integer,
    'DATE_Year': Integer,
    'DATE_FirstDayOfMonth': DATE,
    'DATE_LastDayOfMonth': DATE,
    'DATE_YYYYMM': CHAR(10),
    'DATE_WeekendIndr': CHAR(15)
}

prepare_and_insert(date_table, date_dtypes, 'Date', united_outdoors_engine)

Inserting data into table: Date with chunk size: 125


### Time

In [548]:
time_dtypes = {
    'TIME_ID': Integer,
    'TIME_Hour': Integer,
    'TIME_Minute': Integer,
    'TIME_HourMinute': VARCHAR(10),
}

prepare_and_insert(time_table, time_dtypes, 'Time', united_outdoors_engine)

Inserting data into table: Time with chunk size: 500


### Products

In [ ]:
product_dtypes =  {
    'PRODUCT_SK': Integer,
    'PRODUCT_PRODUCT_ID': Integer,
    'PRODUCT_PRODUCT_SizeUnitMeasureCode': VARCHAR(10),
    'PRODUCT_PRODUCT_WeightUnitMeasureCode': VARCHAR(10),
    'PRODUCT_PRODUCT_SubCategoryID': Integer,
    'PRODUCT_PRODUCTSUBCATEGORY_CategoryID': Integer,
    'PRODUCT_PRODUCT_ModelID': Integer,
    'PRODUCT_PRODUCTMODELILLUSTRATION_IllustrationID': Integer,
    'PRODUCT_PRODUCTPRODUCTPHOTO_PhotoID': Integer,
    'PRODUCT_PMPDC_DescriptionID': Integer,
    'PRODUCT_PMPDC_CultureID': Integer,
    'PRODUCT_PRODUCT_Name': VARCHAR(150),
    'PRODUCT_PRODUCT_Number': VARCHAR(25),
    'PRODUCT_PRODUCT_MakeFlag': Integer,
    'PRODUCT_PRODUCT_FinishedGoodsFlag': Integer,
    'PRODUCT_PRODUCT_Color': VARCHAR(25),
    'PRODUCT_PRODUCT_SafetyStockLevel': Integer,
    'PRODUCT_PRODUCT_ReorderPoint': Integer,
    'PRODUCT_PRODUCT_ReorderLevel': Integer,
    'PRODUCT_PRODUCT_StandardCost': DECIMAL(8, 4),
    'PRODUCT_PRODUCT_UnitPrice': DECIMAL(19, 4),
    'PRODUCT_PRODUCT_ListPrice': DECIMAL(19, 4),
    'PRODUCT_PRODUCT_Size': VARCHAR(50),
    'PRODUCT_PRODUCT_SizeUnitMeasureName': VARCHAR(50),
    'PRODUCT_PRODUCT_WeightUnitMeasureName': VARCHAR(50),
    'PRODUCT_Color': VARCHAR(50),
    'PRODUCT_PRODUCT_Weight': DECIMAL(8, 2),
    'PRODUCT_PRODUCT_DaysToManufacture': Integer,
    'PRODUCT_PRODUCT_ProductLine': VARCHAR(10),
    'PRODUCT_PRODUCT_Class': VARCHAR(10),
    'PRODUCT_PRODUCT_Style': VARCHAR(10),
    'PRODUCT_PRODUCTSUBCATEGORY_SubCategory': VARCHAR(50),
    'PRODUCT_PRODUCTCATEGORY_Category': VARCHAR(50),
    'PRODUCT_CATEGORY_Picture': LargeBinary,
    'PRODUCT_PRODUCTMODEL_Name': VARCHAR(100),
    'PRODUCT_PRODUCTMODEL_CatalogDescription': XML,
    'PRODUCT_PRODUCTMODEL_Instructions': XML,
    'PRODUCT_ILLUSTRATION_Diagram': XML,
    'PRODUCT_CULTURE_Name': VARCHAR(50),
    'PRODUCT_PRODUCTDESCRIPTION_Desc': NVARCHAR,
    'PRODUCT_PRODUCTPRODUCTPHOTO_Primary': Integer,
    'PRODUCT_PRODUCTPHOTO_ThumbnailPhoto': LargeBinary,
    'PRODUCT_PRODUCTPHOTO_ThumbnailPhotoHexString': NVARCHAR,
    'PRODUCT_PRODUCTPHOTO_ThumbnailPhotoFileName': VARCHAR(50),
    'PRODUCT_PRODUCTPHOTO_LargePhoto': LargeBinary,
    'PRODUCT_PRODUCTPHOTO_LargePhotoHexString': NVARCHAR,
    'PRODUCT_PRODUCTPHOTO_LargePhotoFileName': VARCHAR(50),
    'PRODUCT_PRODUCT_SellStartDate': DATE,
    'PRODUCT_DATE_SellStartDateFK': Integer,
    'PRODUCT_PRODUCT_SellEndDate': DATE,
    'PRODUCT_DATE_SellEndDateFK': Integer,
    'PRODUCT_PRODUCT_DiscountedDate': DATE,
    'PRODUCT_DATE_DiscountedDateFK': Integer,
    'PRODUCT_PRODUCT_Discontinued': Integer,
}

products_nk_sk_dict = prepare_and_insert_return_sk(product_combined, product_dtypes, 'Product', united_outdoors_engine, 'PRODUCT_PRODUCT_ID')

### BillOfMaterial

In [549]:
billofmaterial_dtypes = {
    'BILLOFMATERIAL_BILLOFMATERIAL_BillOfMaterialID': Integer,
    'BILLOFMATERIAL_BILLOFMATERIAL_ProductAssemblyID': Integer,
    'BILLOFMATERIAL_BILLOFMATERIAL_ComponentID': Integer,
    'BILLOFMATERIAL_BILLOFMATERIAL_StartDate': DATE,
    'BILLOFMATERIAL_BILLOFMATERIAL_EndDate': DATE,
    'BILLOFMATERIAL_BILLOFMATERIAL_UnitMeasureCode': CHAR(3),
    'BILLOFMATERIAL_BILLOFMATERIAL_BOMLevel': Integer,
    'BILLOFMATERIAL_BILLOFMATERIAL_PerAssemblyQty': DECIMAL(8,2),
    'BILLOFMATERIAL_UNITMEASURE_Name': NVARCHAR(50)
}

prepare_and_insert(billofmaterials, billofmaterial_dtypes, "BillOfMaterial", united_outdoors_engine)

Inserting data into table: BillOfMaterial with chunk size: 222


### Document

In [550]:
document_dtypes = {
    'DOCUMENT_DOCUMENT_DocumentNode': NVARCHAR,
    'DOCUMENT_DOCUMENT_DocumentLevel': Integer,
    'DOCUMENT_DOCUMENT_Title': NVARCHAR(50),
    'DOCUMENT_DOCUMENT_Owner': Integer,
    'DOCUMENT_DOCUMENT_FolderFlag': BIT,
    'DOCUMENT_DOCUMENT_FileName': NVARCHAR(400),
    'DOCUMENT_DOCUMENT_FileExtension': NVARCHAR(8),
    'DOCUMENT_DOCUMENT_Revision': CHAR(5),
    'DOCUMENT_DOCUMENT_ChangeNumber': Integer,
    'DOCUMENT_DOCUMENT_Status': Integer,
    'DOCUMENT_DOCUMENT_DocumentSummary': NVARCHAR,
    'DOCUMENT_DOCUMENT_Document': LargeBinary,
    'DOCUMENT_PRODUCTDOCUMENT_ProductID': Integer,
}

prepare_and_insert(documents, document_dtypes, "Document", united_outdoors_engine, {'DOCUMENT_PRODUCTDOCUMENT_ProductID': products_nk_sk_dict})

NameError: name 'products_nk_sk_dict' is not defined

### WorkOrder

In [ ]:
workorder_dtypes = {
    'WORKORDER_WORKORDER_WorkOrderID': Integer,
    'WORKORDER_WORKORDER_ProductID': Integer,
    'WORKORDER_WORKORDER_OrderQty': Integer,
    'WORKORDER_WORKORDER_StockedQty': Integer,
    'WORKORDER_WORKORDER_ScrappedQty': Integer,
    'WORKORDER_WORKORDER_StartDate': DATE,
    'WORKORDER_WORKORDER_EndDate': DATE,
    'WORKORDER_WORKORDER_DueDate': DATE,
    'WORKORDER_WORKORDER_ScrapReasonID': Integer,
    'WORKORDER_WORKORDERINGROUTING_ProductID': Integer,
    'WORKORDER_WORKORDERINGROUTING_OperationSequence': Integer,
    'WORKORDER_WORKORDERINGROUTING_LocationID': Integer,
    'WORKORDER_WORKORDERINGROUTING_ScheduledStartDate': DATE,
    'WORKORDER_WORKORDERINGROUTING_ScheduledEndDate': DATE,
    'WORKORDER_WORKORDERINGROUTING_ActualStartDate': DATE,
    'WORKORDER_WORKORDERINGROUTING_ActualEndDate': DATE,
    'WORKORDER_WORKORDERINGROUTING_ActualResourcesHrs': DECIMAL(9,4),
    'WORKORDER_WORKORDERINGROUTING_PlannedCost': MONEY,
    'WORKORDER_WORKORDERINGROUTING_ActualCost': MONEY,
    'WORKORDER_SCRAPREASON_ScrapReasonID': Integer,
    'WORKORDER_SCRAPREASON_Name': NVARCHAR(50)
}

prepare_and_insert(workorders, workorder_dtypes, "WorkOrder", united_outdoors_engine, {'WORKORDER_WORKORDER_ProductID': products_nk_sk_dict})

Inserting data into table: WorkOrder with chunk size: 105


### Departments

In [ ]:
departments_dtypes = {
    'DEPARTMENT_DEPARTMENT_DeptID': Integer,
    'DEPARTMENT_DEPARTMENT_DeptName': String(100),
    'DEPARTMENT_DEPARTMENT_GroupName': String(100),
    'DEPARTMENT_DEPARTMENT_DeptHeadID': Integer,
    'DEPARTMENT_source_database': String(100)
}

# TODO dept_head_id needs to refer to an employee
departments_nk_sk_dict = prepare_and_insert_return_sk(departments, departments_dtypes, 'Department', united_outdoors_engine, 'DEPARTMENT_DEPARTMENT_DeptID', { 'DEPARTMENT_DEPARTMENT_DeptHeadID' : {}})

Replacing natural keys with surrogate keys for column: DEPARTMENT_DEPARTMENT_DeptHeadID
Inserting data into table: Department with chunk size: 400


### Employees

In [ ]:
employees_dtypes = {
    'EMPLOYEE_EMPLOYEE_EmployeeID': Integer,
    'EMPLOYEE_EMPLOYEE_DepartmentID': Integer,
    'EMPLOYEE_EMPLOYEE_ManagerID': Integer,
    'EMPLOYEE_EMPLOYEETERRITORIES_TerritoryID': Integer,
    'EMPLOYEE_EMPLOYEE_Emp_Fname': NVARCHAR(255),
    'EMPLOYEE_EMPLOYEE_Emp_Lname': NVARCHAR(255),
    'EMPLOYEE_EMPLOYEE_Street': NVARCHAR(150),
    'EMPLOYEE_EMPLOYEE_City': NVARCHAR(100),
    'EMPLOYEE_EMPLOYEE_State': CHAR(1),
    'EMPLOYEE_EMPLOYEE_Zip_Code': CHAR(5),
    'EMPLOYEE_EMPLOYEE_Phone': NVARCHAR(20),
    'EMPLOYEE_EMPLOYEE_Status': CHAR(1),
    'EMPLOYEE_EMPLOYEE_SS_Number': Integer,
    'EMPLOYEE_EMPLOYEE_Salary': Integer,
    'EMPLOYEE_EMPLOYEE_Start_Date': DATE,
    'EMPLOYEE_EMPLOYEE_Termination': DATE,
    'EMPLOYEE_EMPLOYEE_Birth_Date': DATE,
    'EMPLOYEE_EMPLOYEE_Bene_Health_Ins': CHAR(1),
    'EMPLOYEE_EMPLOYEE_Bene_Life_Ins': CHAR(1),
    'EMPLOYEE_EMPLOYEE_Bene_Day_Care': CHAR(1),
    'EMPLOYEE_EMPLOYEE_Sex': CHAR(1),
    'EMPLOYEE_BONUS_Bonus_Date': DATE,
    'EMPLOYEE_BONUS_Bonus_Amount': Integer,
    'EMPLOYEE_EMPLOYEES_Title': NVARCHAR(50),
    'EMPLOYEE_EMPLOYEES_TitleOfCourtesy': NVARCHAR(50),
    'EMPLOYEE_EMPLOYEES_HireDate': DATE,
    'EMPLOYEE_EMPLOYEES_HomePhone': NVARCHAR(20),
    'EMPLOYEE_EMPLOYEES_Extension': Integer,
    'EMPLOYEE_EMPLOYEES_PhotoHexString': String,
    'EMPLOYEE_EMPLOYEES_PhotoPath': NVARCHAR(255),
    'EMPLOYEE_EMPLOYEES_Notes': String,
}

employees_nk_sk_dict = prepare_and_insert_return_sk(employees, employees_dtypes, 'Employee', united_outdoors_engine, 'EMPLOYEE_EMPLOYEE_EmployeeID', { 'EMPLOYEE_EMPLOYEE_DepartmentID' : departments_nk_sk_dict, 'EMPLOYEE_EMPLOYEE_ManagerID' : departments_nk_sk_dict})

Replacing natural keys with surrogate keys for column: EMPLOYEE_EMPLOYEE_DepartmentID
Replacing natural keys with surrogate keys for column: EMPLOYEE_EMPLOYEE_ManagerID
Inserting data into table: Employee with chunk size: 64


### BusinessEntities

In [ ]:
businessentities_dtypes = {
    'BUSINESSENTITY_BUSINESSENTITY_BusinessEntityID': Integer,
    'BUSINESSENTITY_BUSINESSENTITYCONTACT_PersonID': Integer,
    'BUSINESSENTITY_CONTACTTYPE_ContactTypeID': Integer,
    'BUSINESSENTITY_CONTACTTYPE_Name': String(100)
}

businessentities_nk_sk_dict = prepare_and_insert_return_sk(businessentities, businessentities_dtypes, 'BusinessEntity',united_outdoors_engine, 'BUSINESSENTITY_BUSINESSENTITY_BusinessEntityID')

Inserting data into table: BusinessEntity with chunk size: 500


### People

In [ ]:
people_dtypes = {
    'PERSON_PERSON_BusinessEntityID': Integer,
    'PERSON_PERSON_PersonType': String(2),
    'PERSON_PERSON_NameStyle': BIT,
    'PERSON_PERSON_Title': String(100),
    'PERSON_PERSON_FirstName': String(100),
    'PERSON_PERSON_MiddleName': String(100),
    'PERSON_PERSON_LastName': String(100),
    'PERSON_PERSON_Suffix': String(100),
    'PERSON_PERSON_EmailPromotion': Integer,
    'PERSON_PERSON_AdditionalContactInfo': XML,
    'PERSON_PERSON_Demographics': XML,
    'PERSON_PERSONPHONE_PhoneNumber': String(100),
    'PERSON_PHONENUMBERTYPE_PhoneNumberTypeID': Integer,
    'PERSON_PHONENUMBERTYPE_Name': String(100),
    'PERSON_EMAILADDRESS_EmailAddressID': Integer,
    'PERSON_EMAILADDRESS_EmailAddress': String(100),
    'PERSON_PASSWORD_PasswordHash': LargeBinary,
    'PERSON_PASSWORD_PasswordSalt': LargeBinary
}

# Convert the 'PERSON_PASSWORD_PasswordHash' and 'PERSON_PASSWORD_PasswordSalt' columns to bytes
people['PERSON_PASSWORD_PasswordHash'] = people['PERSON_PASSWORD_PasswordHash'].apply(lambda x: x.encode('utf-8') if isinstance(x, str) else x)
people['PERSON_PASSWORD_PasswordSalt'] = people['PERSON_PASSWORD_PasswordSalt'].apply(lambda x: x.encode('utf-8') if isinstance(x, str) else x)

prepare_and_insert(people, people_dtypes, 'Person',united_outdoors_engine, { 'PERSON_PERSON_BusinessEntityID' : businessentities_nk_sk_dict})

Replacing natural keys with surrogate keys for column: PERSON_PERSON_BusinessEntityID
Inserting data into table: Person with chunk size: 111


### Updating the BusinessEntity table
replacing the natural keys with the surrogate keys for the PersonID column

In [ ]:
prepare_and_update('BusinessEntity', united_outdoors_engine, { 'BUSINESSENTITY_BUSINESSENTITYCONTACT_PersonID' : businessentities_nk_sk_dict})

Updating data in table: BusinessEntity


### BusinessEntityAddresses

In [ ]:
businessentityaddresses_dtypes = {
    'BUSINESSENTITYADDRESS_BUSINESSENTITYADDRESS_BusinessEntityID': Integer,
    'BUSINESSENTITYADDRESS_ADDRESSTYPE_AddressTypeID': Integer,
    'BUSINESSENTITYADDRESS_ADDRESSTYPE_Name': String(100),
    'BUSINESSENTITYADDRESS_ADDRESS_AddressID': Integer,
    'BUSINESSENTITYADDRESS_ADDRESS_AddressLine1': String(100),
    'BUSINESSENTITYADDRESS_ADDRESS_AddressLine2': String(100),
    'BUSINESSENTITYADDRESS_ADDRESS_City': String(100),
    'BUSINESSENTITYADDRESS_ADDRESS_POSTALCODE': String(100),
    'BUSINESSENTITYADDRESS_ADDRESS_SpatialLocation': VARCHAR,
    'BUSINESSENTITYADDRESS_ADDRESS_StateProvinceID': Integer
}

# TODO the StateProvinceID needs to refer to sk of Territory
prepare_and_insert(businessentityaddresses, businessentityaddresses_dtypes, 'BusinessEntityAddress',united_outdoors_engine, { 'BUSINESSENTITYADDRESS_BUSINESSENTITYADDRESS_BusinessEntityID' : businessentities_nk_sk_dict, 'BUSINESSENTITYADDRESS_ADDRESS_StateProvinceID' : {}})

Replacing natural keys with surrogate keys for column: BUSINESSENTITYADDRESS_BUSINESSENTITYADDRESS_BusinessEntityID
Replacing natural keys with surrogate keys for column: BUSINESSENTITYADDRESS_ADDRESS_StateProvinceID
Inserting data into table: BusinessEntityAddress with chunk size: 200


### Inventory

In [ ]:
inventory_dtypes =  {
    'INVENTORY_sk' : Integer,
    'INVENTORY_PRODUCT_ID' : Integer,
    'INVENTORY_LOCATION_ID' : Integer,
    'INVENTORY_Shelf' : VARCHAR(5),
    'INVENTORY_Bin' : Integer,
    'INVENTORY_Quantity' : Integer,
}

prepare_and_insert(inventory_combined, inventory_dtypes, 'Inventory', united_outdoors_engine)

Inserting data into table: Inventory with chunk size: 400


### JobCandidate

In [ ]:
jobcandidates_dtypes = {
    'JOBCANDIDATE_JOBCANDIDATE_JobCandidateID': Integer,
    'JOBCANDIDATE_JOBCANDIDATE_BusinessEntityID': Integer,
    'JOBCANDIDATE_JOBCANDIDATE_Resume': XML
}

prepare_and_insert(jobcandidates, jobcandidates_dtypes, 'JobCandidate',united_outdoors_engine, { 'JOBCANDIDATE_JOBCANDIDATE_BusinessEntityID' : businessentities_nk_sk_dict})

Replacing natural keys with surrogate keys for column: JOBCANDIDATE_JOBCANDIDATE_BusinessEntityID
Inserting data into table: JobCandidate with chunk size: 666


### EmployeeDepartmentHistory

In [ ]:
employeedepartmenthistories_dtypes = {
    'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_BusinessEntityID': Integer,
    'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_DepartmentID': Integer,
    'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_ShiftID': Integer,
    'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_StartDate': DATE,
    'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_EndDate': DATE
}

prepare_and_insert(employeedepartmenthistories, employeedepartmenthistories_dtypes, 'EmployeeDepartmentHistory',united_outdoors_engine, { 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_BusinessEntityID' : businessentities_nk_sk_dict, 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_DepartmentID' : departments_nk_sk_dict, 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_ShiftID' : {}})

Replacing natural keys with surrogate keys for column: EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_BusinessEntityID
Replacing natural keys with surrogate keys for column: EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_DepartmentID
Replacing natural keys with surrogate keys for column: EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_ShiftID
Inserting data into table: EmployeeDepartmentHistory with chunk size: 400


### EmployeePayHistory

In [ ]:
employeepayhistories_dtypes = {
    'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_BusinessEntityID': Integer,
    'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_RateChangeDate': DATE,
    'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_Rate': MONEY,
    'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_PayFrequency': Integer
}

prepare_and_insert(employeepayhistories, employeepayhistories_dtypes, 'EmployeePayHistory',united_outdoors_engine, { 'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_BusinessEntityID' : businessentities_nk_sk_dict})

Replacing natural keys with surrogate keys for column: EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_BusinessEntityID
Inserting data into table: EmployeePayHistory with chunk size: 500


### Shift

In [ ]:
shifts_dtypes = {
    'SHIFT_SHIFT_ShiftID': Integer,
    'SHIFT_SHIFT_Name': String(100),
    'SHIFT_SHIFT_StartTime': TIME,
    'SHIFT_SHIFT_EndTime': TIME
}

prepare_and_insert(shifts, shifts_dtypes, 'Shift',united_outdoors_engine)

Inserting data into table: Shift with chunk size: 500


### SalesPerson

In [ ]:
salespeople_dtypes = {
    'SALESPERSON_SALESPERSON_BusinessEntityID': Integer,
    'SALESPERSON_SALESPERSON_TerritoryID': Integer,
    'SALESPERSON_SALESPERSON_SalesQuota': MONEY,
    'SALESPERSON_SALESPERSON_Bonus': MONEY,
    'SALESPERSON_SALESPERSON_CommissionPct': DECIMAL(8,4),
    'SALESPERSON_SALESPERSON_SalesYTD': MONEY,
    'SALESPERSON_SALESPERSON_SalesLastYear': MONEY
}

prepare_and_insert(salespeople, salespeople_dtypes, 'SalesPerson',united_outdoors_engine, { 'SALESPERSON_SALESPERSON_BusinessEntityID' : businessentities_nk_sk_dict, 'SALESPERSON_SALESPERSON_TerritoryID' : {}})

Replacing natural keys with surrogate keys for column: SALESPERSON_SALESPERSON_BusinessEntityID
Replacing natural keys with surrogate keys for column: SALESPERSON_SALESPERSON_TerritoryID
Inserting data into table: SalesPerson with chunk size: 285


### ProductVendor

In [ ]:
productvendors_dtypes = {
    'PRODUCTVENDOR_PRODUCTVENDOR_ProductID': Integer,
    'PRODUCTVENDOR_PRODUCTVENDOR_BusinessEntityID': Integer,
    'PRODUCTVENDOR_PRODUCTVENDOR_AverageLeadTime': Integer,
    'PRODUCTVENDOR_PRODUCTVENDOR_StandardPrice': MONEY,
    'PRODUCTVENDOR_PRODUCTVENDOR_LastReceiptCost': MONEY,
    'PRODUCTVENDOR_PRODUCTVENDOR_LastReceiptDate': DATE,
    'PRODUCTVENDOR_PRODUCTVENDOR_MinOrderQty': Integer,
    'PRODUCTVENDOR_PRODUCTVENDOR_MaxOrderQty': Integer,
    'PRODUCTVENDOR_PRODUCTVENDOR_OnOrderQty': Integer,
    'PRODUCTVENDOR_PRODUCTVENDOR_UnitMeasureCode': CHAR(3)
}

prepare_and_insert(productvendors, productvendors_dtypes, 'ProductVendor', united_outdoors_engine, { 'PRODUCTVENDOR_PRODUCTVENDOR_ProductID' : products_nk_sk_dict, 'PRODUCTVENDOR_PRODUCTVENDOR_BusinessEntityID' : businessentities_nk_sk_dict, 'PRODUCTVENDOR_PRODUCTVENDOR_UnitMeasureCode' : {}})

Replacing natural keys with surrogate keys for column: PRODUCTVENDOR_PRODUCTVENDOR_ProductID
Replacing natural keys with surrogate keys for column: PRODUCTVENDOR_PRODUCTVENDOR_BusinessEntityID
Replacing natural keys with surrogate keys for column: PRODUCTVENDOR_PRODUCTVENDOR_UnitMeasureCode
Inserting data into table: ProductVendor with chunk size: 200


### CustomerDemographics

In [ ]:
customerdemographics_dtypes = {
    'CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerTypeID': NVARCHAR(10),
    'CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerDesc': String(100)
}

customerdemographics_nk_sk_dict = prepare_and_insert_return_sk(customerdemographics, customerdemographics_dtypes, 'CustomerDemographic',united_outdoors_engine, 'CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerTypeID')

Inserting data into table: CustomerDemographic with chunk size: 1000


### CustomerCustomerDemo

In [ ]:
customercustomerdemos_dtypes = {
    'CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerID': NVARCHAR(10),
    'CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerTypeID': NVARCHAR(10)
}

# TODO how to handle fk? the nk is nvarchar, not an integer

prepare_and_insert(customercustomerdemos, customercustomerdemos_dtypes, 'CustomerCustomerDemo',united_outdoors_engine, { 'CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerTypeID' : customerdemographics_nk_sk_dict, 'CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerID' : {}})

Replacing natural keys with surrogate keys for column: CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerTypeID
Replacing natural keys with surrogate keys for column: CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerID
Inserting data into table: CustomerCustomerDemo with chunk size: 1000


### SalesTerritoryHistory

In [ ]:
salesterritoryhistories_dtypes = {
    'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_BusinessEntityID': Integer,
    'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_TerritoryID': Integer,
    'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_StartDate': DATE,
    'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_EndDate': DATE
}

prepare_and_insert(salesterritoryhistories, salesterritoryhistories_dtypes, 'SalesTerritoryHistory',united_outdoors_engine, { 'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_BusinessEntityID' : businessentities_nk_sk_dict, 'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_TerritoryID' : {}})

Replacing natural keys with surrogate keys for column: SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_BusinessEntityID
Replacing natural keys with surrogate keys for column: SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_TerritoryID
Inserting data into table: SalesTerritoryHistory with chunk size: 500


### ProductListPriceHistory

In [ ]:
productlistpricehistories_dtypes = {
    'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ProductID': Integer,
    'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_StartDate': DATE,
    'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_EndDate': DATE,
    'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ListPrice': MONEY
}

prepare_and_insert(productlistpricehistories, productlistpricehistories_dtypes, 'ProductListPriceHistory',united_outdoors_engine, { 'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ProductID' : products_nk_sk_dict})

Replacing natural keys with surrogate keys for column: PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ProductID
Inserting data into table: ProductListPriceHistory with chunk size: 500


### ProductCostHistory

In [ ]:
productcosthistories_dtypes = {
    'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_ProductID': Integer,
    'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_StartDate': DATE,
    'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_EndDate': DATE,
    'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_StandardCost': MONEY
}

prepare_and_insert(productcosthistories, productcosthistories_dtypes, 'ProductCostHistory', united_outdoors_engine, { 'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_ProductID' : {}})

Replacing natural keys with surrogate keys for column: PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_ProductID
Inserting data into table: ProductCostHistory with chunk size: 500


### ShoppingCartItem

In [ ]:
shoppingcartitems_dtypes = {
    'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartItemID': Integer,
    'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartID': Integer,
    'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ProductID': Integer,
    'SHOPPINGCARTITEM_SHOPPINGCARTITEM_Quantity': Integer,
    'SHOPPINGCARTITEM_SHOPPINGCARTITEM_DateCreated': DATE
}

prepare_and_insert(shoppingcartitems, shoppingcartitems_dtypes, 'ShoppingCartItem', united_outdoors_engine, { 'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ProductID' : products_nk_sk_dict, 'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartID' : {}})

Replacing natural keys with surrogate keys for column: SHOPPINGCARTITEM_SHOPPINGCARTITEM_ProductID
Replacing natural keys with surrogate keys for column: SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartID
Inserting data into table: ShoppingCartItem with chunk size: 400


### SalesPersonQuotaHistory

In [ ]:
salespersonquotahistories_dtypes = {
    'SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_BusinessEntityID': Integer,
    'SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_QuotaDate': DATE,
    'SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_SalesQuota': MONEY
}

prepare_and_insert(salespersonquotahistories, salespersonquotahistories_dtypes, 'SalesPersonQuotaHistory', united_outdoors_engine, { 'SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_BusinessEntityID' : businessentities_nk_sk_dict})

Replacing natural keys with surrogate keys for column: SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_BusinessEntityID
Inserting data into table: SalesPersonQuotaHistory with chunk size: 666


### Customers

In [ ]:
customers_dtypes = {
    'CUSTOMER_CUSTOMERS_ID': Integer,
    'CUSTOMER_CUSTOMER_PersonID': Integer,
    'CUSTOMER_CUSTOMER_StoreID': Integer,
    'CUSTOMER_CUSTOMER_TerritoryID': Integer,
    'CUSTOMER_CUSTOMER_AccountNumber': CHAR(10),
    'CUSTOMER_CUSTOMERS_CompanyName': NVARCHAR(100),
    'CUSTOMER_CUSTOMERS_ContactName': NVARCHAR(255),
    'CUSTOMER_CUSTOMERS_ContactTitle': NVARCHAR(100),
    'CUSTOMER_CUSTOMERS_Address': NVARCHAR(255),
    'CUSTOMER_CUSTOMERS_City': NVARCHAR(100),
    'CUSTOMER_CUSTOMERS_Region': NVARCHAR(50),
    'CUSTOMER_CUSTOMERS_PostalCode': NVARCHAR(20),
    'CUSTOMER_CUSTOMERS_Country': NVARCHAR(150),
    'CUSTOMER_CUSTOMERS_Phone': NVARCHAR(24),
    'CUSTOMER_CUSTOMERS_Fax': NVARCHAR(24),
    'CUSTOMER_CUSTOMER_Fname': NVARCHAR(255),
    'CUSTOMER_CUSTOMER_Lname': NVARCHAR(255),
    'CUSTOMER_CUSTOMER_State': CHAR(2),
    'CUSTOMER_STORE_Name': NVARCHAR(50),
    'CUSTOMER_STORE_SalesPersonID': Integer,
    'CUSTOMER_STORE_Demographics': XML,
    'CUSTOMER_SALESTERRITORY_Name': NVARCHAR(50),
    'CUSTOMER_SALESTERRITORY_CountryRegionCode': NVARCHAR(3),
    'CUSTOMER_SALESTERRITORY_Group': NVARCHAR(50),
    'CUSTOMER_SALESTERRITORY_SalesYTD': MONEY,
    'CUSTOMER_SALESTERRITORY_SalesLastYear': MONEY,
    'CUSTOMER_SALESTERRITORY_CostYTD': MONEY,
    'CUSTOMER_SALESTERRITORY_CostLastYear': MONEY
}

customers_nk_sk_dict = prepare_and_insert_return_sk(customers, customers_dtypes, 'Customer', united_outdoors_engine, 'CUSTOMER_CUSTOMERS_ID', {'CUSTOMER_CUSTOMER_PersonID': {}, 'CUSTOMER_CUSTOMER_StoreID': {}, 'CUSTOMER_CUSTOMER_TerritoryID': {}, 'CUSTOMER_STORE_SalesPersonID': {}, 'CUSTOMER_SALESTERRITORY_CountryRegionCode': {}})

Replacing natural keys with surrogate keys for column: CUSTOMER_CUSTOMER_PersonID
Replacing natural keys with surrogate keys for column: CUSTOMER_CUSTOMER_StoreID
Replacing natural keys with surrogate keys for column: CUSTOMER_CUSTOMER_TerritoryID
Inserting data into table: Customer with chunk size: 111


### SpecialOffer

In [ ]:
specialoffers_dtypes = {
    'SPECIALOFFER_SPECIALOFFER_ID': Integer,
    'SPECIALOFFER_SPECIALOFFER_Description': String,
    'SPECIALOFFER_SPECIALOFFER_DiscountPCT': Integer,
    'SPECIALOFFER_SPECIALOFFER_Type': NVARCHAR(100),
    'SPECIALOFFER_SPECIALOFFER_Category': NVARCHAR(100),
    'SPECIALOFFER_SPECIALOFFER_StartDate': DATE,
    'SPECIALOFFER_SPECIALOFFER_EndDate': DATE,
    'SPECIALOFFER_SPECIALOFFER_MinQty': Integer,
    'SPECIALOFFER_SPECIALOFFER_MaxQty': Integer,
    'SPECIALOFFER_SPECIALOFFERPRODUCT_ProductID': Integer
}

specialoffer_nk_sk_dict = prepare_and_insert_return_sk(specialoffers, specialoffers_dtypes, 'SpecialOffer', united_outdoors_engine, 'SPECIALOFFER_SPECIALOFFER_ID', {'SPECIALOFFER_SPEIALOFFERPRODUCT_ProductID': products_nk_sk_dict})

Inserting data into table: SpecialOffer with chunk size: 200


### TransactionHistory

In [ ]:
transactionhistories_dtypes = {
    'TRANSACTIONHISTORY_TRANSACTIONHISTORY_TransactionID': Integer,
    'TRANSACTIONHISTORY_TRANSACTIONHISTORY_ProductID': Integer,
    'TRANSACTIONHISTORY_TRANSACTIONHISTORY_ReferenceOrderID': Integer,
    'TRANSACTIONHISTORY_TRANSACTIONHISTORY_ReferenceOrderLineID': Integer,
    'TRANSACTIONHISTORY_TRANSACTIONHISTORY_TransactionDate': DATE,
    'TRANSACTIONHISTORY_TRANSACTIONHISTORY_TransactionType': CHAR(1),
    'TRANSACTIONHISTORY_TRANSACTIONHISTORY_Quantity': Integer,
    'TRANSACTIONHISTORY_TRANSACTIONHISTORY_ActualCost': MONEY,
}

prepare_and_insert(transactionhistories, transactionhistories_dtypes, 'TransactionHistory', united_outdoors_engine, { 'TRANSACTIONHISTORY_TRANSACTIONHISTORY_ProductID' : products_nk_sk_dict})

Replacing natural keys with surrogate keys for column: TRANSACTIONHISTORY_TRANSACTIONHISTORY_ProductID
Inserting data into table: TransactionHistory with chunk size: 250


### CreditCard

In [ ]:
creditcards_dtypes = {
    'CREDITCARD_CREDITCARD_ID': Integer,
    'CREDITCARD_CREDITCARD_CardType': NVARCHAR(100),
    'CREDITCARD_CREDITCARD_CardNumber': BigInteger,
    'CREDITCARD_CREDITCARD_ExpMonth': Integer,
    'CREDITCARD_CREDITCARD_ExpYear': Integer,
    'CREDITCARD_PERSONCREDITCARD_BusinessEntityID': Integer
}

creditcards_nk_sk_dict = prepare_and_insert_return_sk(creditcards, creditcards_dtypes, 'CreditCard', united_outdoors_engine, 'CREDITCARD_CREDITCARD_ID')

Inserting data into table: CreditCard with chunk size: 333


### Supplier

In [ ]:
suppliers_dtypes = {
    'SUPPLIER_SUPPLIERS_SupplierID': Integer,
    'SUPPLIER_SUPPLIERS_CompanyName': NVARCHAR(100),
    'SUPPLIER_SUPPLIERS_ContactName': NVARCHAR(255),
    'SUPPLIER_SUPPLIERS_ContactTitle': NVARCHAR(100),
    'SUPPLIER_SUPPLIERS_Address': NVARCHAR(150),
    'SUPPLIER_SUPPLIERS_City': NVARCHAR(50),
    'SUPPLIER_SUPPLIERS_Region': NVARCHAR(50),
    'SUPPLIER_SUPPLIERS_PostalCode': NVARCHAR(20),
    'SUPPLIER_SUPPLIERS_Country': NVARCHAR(100),
    'SUPPLIER_SUPPLIERS_Phone': VARCHAR(20),
    'SUPPLIER_SUPPLIERS_Fax': NVARCHAR(30),
    'SUPPLIER_SUPPLIERS_HomePage': NVARCHAR(255)
}

prepare_and_insert(suppliers, suppliers_dtypes, 'Supplier', united_outdoors_engine)

Inserting data into table: Supplier with chunk size: 166


### Currency

In [ ]:
currencies_dtypes = {
    'CURRENCY_CURRENCY_CurrencyCode': NVARCHAR(10),
    'CURRENCY_CURRENCY_Name': NVARCHAR(100),
    'CURRENCY_COUNTRYREGIONCURRENCY_CountryRegionCode': NVARCHAR(10),
    'CURRENCY_CURRENCYRATE_CurrencyRateID': Integer,
    'CURRENCY_CURRENCYRATE_CurrencyRateDate': DATE,
    'CURRENCY_CURRENCYRATE_FromCurrencyCode': NVARCHAR(10),
    'CURRENCY_CURRENCYRATE_ToCurrencyCode': NVARCHAR(10),
    'CURRENCY_CURRENCYRATE_AverageRate': Integer,
    'CURRENCY_CURRENCYRATE_EndOfDayRate': Integer
}

prepare_and_insert(currencies, currencies_dtypes, 'Currency', united_outdoors_engine, {'CURRENCY_CURRENCYRATE_CurrencyRateID': {}, 'CURRENCY_CURRENCYRATE_FromCurrencyCode': {}, 'CURRENCY_CURRENCYRATE_ToCurrencyCode': {}})

Replacing natural keys with surrogate keys for column: CURRENCY_CURRENCYRATE_CurrencyRateID
Replacing natural keys with surrogate keys for column: CURRENCY_CURRENCYRATE_FromCurrencyCode
Replacing natural keys with surrogate keys for column: CURRENCY_CURRENCYRATE_ToCurrencyCode
Inserting data into table: Currency with chunk size: 222


### Territory

In [ ]:
territories_dtypes = {
    'TERRITORY_REGION_RegionID': Integer,
    'TERRITORY_REGION_RegionDescription': String,
    'TERRITORY_TERRITORIES_TerritoryID': Integer,
    'TERRITORY_TERRITORIES_TerritoryDescription': String,
    'TERRITORY_REGION_Region': NVARCHAR(50),
    'TERRITORY_STATE_StateID': NVARCHAR(10),
    'TERRITORY_STATE_StateName': NVARCHAR(100),
    'TERRITORY_STATE_StateCapital': NVARCHAR(50),
    'TERRITORY_STATE_Country': NVARCHAR(50),
    'TERRITORY_COUNTRYREGION_CountryRegionCode': NVARCHAR(10),
    'TERRITORY_COUNTRYREGION_Name': NVARCHAR(100),
    'TERRITORY_STATEPROVINCE_StateProvinceID': Integer,
    'TERRITORY_STATEPROVINCE_StateProvinceCode': NVARCHAR(10),
    'TERRITORY_STATEPROVINCE_IsOnlyStateProvinceFlag': CHAR(1),
    'TERRITORY_STATEPROVINCE_Name': NVARCHAR(100),
    'TERRITORY_SALESTAXRATE_SalesTaxRateID': Integer,
    'TERRITORY_SALESTAXRATE_TaxType': NVARCHAR(50),
    'TERRITORY_SALESTAXRATE_TaxRate': DECIMAL(8,2),
    'TERRITORY_SALESTAXRATE_Name': NVARCHAR(150)
}

prepare_and_insert(territories, territories_dtypes, 'Territory', united_outdoors_engine, {'TERRITORY_TERRITORIES_TerritoryID': {}, 'TERRITORY_STATE_StateID': {}, 'TERRITORY_COUNTRYREGION_CountryRegionCode': {}, 'TERRITORY_STATEPROVINCE_StateProvinceID': {}, 'TERRITORY_SALESTAXRATE_SalesTaxRateID': {}})

Replacing natural keys with surrogate keys for column: TERRITORY_TERRITORIES_TerritoryID
Replacing natural keys with surrogate keys for column: TERRITORY_STATE_StateID
Replacing natural keys with surrogate keys for column: TERRITORY_COUNTRYREGION_CountryRegionCode
Replacing natural keys with surrogate keys for column: TERRITORY_STATEPROVINCE_StateProvinceID
Replacing natural keys with surrogate keys for column: TERRITORY_SALESTAXRATE_SalesTaxRateID
Inserting data into table: Territory with chunk size: 105


### SalesOrderHeader

In [ ]:
salesorderheaders_dtypes = {
    'SALESORDERHEADER_SALESORDERDETAIL_SalesOrderID': Integer,
    'SALESORDERHEADER_SALESORDERDETAIL_SalesOrderDetailID': Integer,
    'SALESORDERHEADER_SALESORDERDETAIL_CarrierTrackingNumber': NVARCHAR(25),
    'SALESORDERHEADER_SALESORDERDETAIL_OrderQty': Integer,
    'SALESORDERHEADER_SALESORDERDETAIL_ProductID': Integer,
    'SALESORDERHEADER_SALESORDERDETAIL_SpecialOfferID': Integer,
    'SALESORDERHEADER_SALESORDERDETAIL_UnitPrice': DECIMAL(8,2),
    'SALESORDERHEADER_SALESORDERDETAIL_UnitPriceDiscount': DECIMAL(8,2),
    'SALESORDERHEADER_SALESORDERDETAIL_LineTotal': NVARCHAR,
    'SALESORDERHEADER_SALESORDERHEADER_RevisionNumber': Integer,
    'SALESORDERHEADER_SALESORDERHEADER_OrderDate': DATE,
    'SALESORDERHEADER_SALESORDERHEADER_DueDate': DATE,
    'SALESORDERHEADER_SALESORDERHEADER_ShipDate': DATE,
    'SALESORDERHEADER_SALESORDERHEADER_Status': CHAR(1),
    'SALESORDERHEADER_SALESORDERHEADER_OnlineOrderFlag': BIT,
    'SALESORDERHEADER_SALESORDERHEADER_SalesOrderNumber': VARCHAR(100),
    'SALESORDERHEADER_SALESORDERHEADER_PurchaseOrderNumber': VARCHAR(25),
    'SALESORDERHEADER_SALESORDERHEADER_AccountNumber': NVARCHAR(15),
    'SALESORDERHEADER_SALESORDERHEADER_CustomerID': Integer,
    'SALESORDERHEADER_SALESORDERHEADER_SalesPersonID': Integer,
    'SALESORDERHEADER_SALESORDERHEADER_TerritoryID': Integer,
    'SALESORDERHEADER_SALESORDERHEADER_BillToAddress': Integer,
    'SALESORDERHEADER_SALESORDERHEADER_ShipToAddress': Integer,
    'SALESORDERHEADER_SALESORDERHEADER_ShipMethodID': Integer,
    'SALESORDERHEADER_SALESORDERHEADER_CreditCardID': Integer,
    'SALESORDERHEADER_SALESORDERHEADER_CreditCardApprovalCode': NVARCHAR(15),
    'SALESORDERHEADER_SALESORDERHEADER_CurrencyRateID': Integer,
    'SALESORDERHEADER_SALESORDERHEADER_SubTotal': DECIMAL(11,2),
    'SALESORDERHEADER_SALESORDERHEADER_TaxAmt': DECIMAL(11,2),
    'SALESORDERHEADER_SALESORDERHEADER_Freight': DECIMAL(11,2),
    'SALESORDERHEADER_SALESORDERHEADER_TotalDue': DECIMAL(11,2),
    'SALESORDERHEADER_SALESORDERHEADER_Comment': NVARCHAR(128),
    'SALESORDERHEADER_SALESORDERHEADERSALESREASON_SalesOrderID': Integer,
    'SALESORDERHEADER_SALESREASON_SalesReasonID': Integer,
    'SALESORDERHEADER_SALESREASON_Name': NVARCHAR(100),
    'SALESORDERHEADER_SALESREASON_ReasonType': NVARCHAR(100),
}

prepare_and_insert(salesorderheaders, salesorderheaders_dtypes, 'SalesOrderHeader', united_outdoors_engine, {'SALESORDERHEADER_SALESORDERDETAIL_ProductID': products_nk_sk_dict,'SALESORDERHEADER_SALESORDERHEADER_CustomerID' : {}, 'SALESORDERHEADER_SALESORDERDETAIL_SpecialOfferID' : specialoffer_nk_sk_dict, 'SALESORDERHEADER_SALESORDERHEADER_CreditCardID': creditcards_nk_sk_dict, 'SALESORDERHEADER_SALESORDERHEADER_CurrencyRateID': {}})


Replacing natural keys with surrogate keys for column: ORDERHEADER_SALESORDERHEADER_CustomerID
Replacing natural keys with surrogate keys for column: ORDERHEADER_SALESORDERDETAIL_SpecialOfferID
Replacing natural keys with surrogate keys for column: ORDERHEADER_SALESORDERHEADER_CreditCardID
Replacing natural keys with surrogate keys for column: ORDERHEADER_SALESORDERHEADER_CurrencyRateID
Inserting data into table: OrderHeader with chunk size: 57


### PurchaseOrderHeader

In [ ]:
purchaseorderheaders_dtypes = {
    'PURCHASEORDERHEADER_PURCHASEORDERHEADER_PurchaseOrderID': Integer,
    'PURCHASEORDERHEADER_PURCHASEORDERHEADER_RevisionNumber': Integer,
    'PURCHASEORDERHEADER_PURCHASEORDERHEADER_Status': Integer,
    'PURCHASEORDERHEADER_PURCHASEORDERHEADER_EmployeeID': Integer,
    'PURCHASEORDERHEADER_PURCHASEORDERHEADER_VendorID': Integer,
    'PURCHASEORDERHEADER_PURCHASEORDERHEADER_ShipMethodID': Integer,
    'PURCHASEORDERHEADER_PURCHASEORDERHEADER_OrderDate': DATE,
    'PURCHASEORDERHEADER_PURCHASEORDERHEADER_ShipDate': DATE,
    'PURCHASEORDERHEADER_PURCHASEORDERHEADER_SubTotal': MONEY,
    'PURCHASEORDERHEADER_PURCHASEORDERHEADER_TaxAmt': MONEY,
    'PURCHASEORDERHEADER_PURCHASEORDERHEADER_Freight': MONEY,
    'PURCHASEORDERHEADER_PURCHASEORDERHEADER_TotalDue': MONEY,
    'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_PurchaseOrderDetailID': Integer,
    'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_DueDate': DATE,
    'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_OrderQty': Integer,
    'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_ProductID': Integer,
    'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_UnitPrice': MONEY,
    'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_LineTotal': NVARCHAR,
    'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_ReceivedQty': DECIMAL(8,2),
    'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_RejectedQty': DECIMAL(8,2),
    'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_StockedQty': DECIMAL(8,2),
    'PURCHASEORDERHEADER_SHIPMETHOD_Name': NVARCHAR(50),
    'PURCHASEORDERHEADER_SHIPMETHOD_ShipBase': MONEY,
    'PURCHASEORDERHEADER_SHIPMETHOD_ShipRate': MONEY,
    'PURCHASEORDERHEADER_VENDOR_AccountNumber': NVARCHAR(50),
    'PURCHASEORDERHEADER_VENDOR_Name': NVARCHAR(50),
    'PURCHASEORDERHEADER_VENDOR_CreditRating': Integer,
    'PURCHASEORDERHEADER_VENDOR_PreferredVendorStatus': BIT,
    'PURCHASEORDERHEADER_VENDOR_ActiveFlag': BIT,
    'PURCHASEORDERHEADER_VENDOR_PurchasingWebServiceURL': NVARCHAR(1024)
}

prepare_and_insert(purchaseorderheaders, purchaseorderheaders_dtypes, 'PurchaseOrderHeader', united_outdoors_engine, {'PURCHASEORDERHEADER_PURCHASEORDERHEADER_EmployeeID': employees_nk_sk_dict, 'PURCHASEORDERHEADER_PURCHASEORDERDETAIL_ProductID': products_nk_sk_dict})

### TransactionHistoryArchive

In [ ]:
transactionhistoryarchives_dtypes = {
    'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_TransactionID': Integer,
    'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_ProductID': Integer,
    'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_ReferenceOrderID': Integer,
    'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_ReferenceOrderLineID': Integer,
    'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_TransactionDate': DATE,
    'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_TransactionType': CHAR(1),
    'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_Quantity': Integer,
    'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_ActualCost': MONEY,
}   

prepare_and_insert(transactionhistoryarchives, transactionhistoryarchives_dtypes, 'TransactionHistoryArchive', united_outdoors_engine, { 'TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_ProductID' : products_nk_sk_dict})

Replacing natural keys with surrogate keys for column: TRANSACTIONHISTORYARCHIVE_TRANSACTIONHISTORYARCHIVE_ProductID
Inserting data into table: TransactionHistoryArchive with chunk size: 250


### ProductReview

In [ ]:
productreviews_dtypes = {
    'PRODUCTREVIEW_PRODUCTREVIEW_ProductReviewID': Integer,
    'PRODUCTREVIEW_PRODUCTREVIEW_ProductID': Integer,
    'PRODUCTREVIEW_PRODUCTREVIEW_ReviewerName': NVARCHAR(50),
    'PRODUCTREVIEW_PRODUCTREVIEW_ReviewDate': DATE,
    'PRODUCTREVIEW_PRODUCTREVIEW_EmailAddress': NVARCHAR(50),
    'PRODUCTREVIEW_PRODUCTREVIEW_Rating': Integer,
    'PRODUCTREVIEW_PRODUCTREVIEW_Comments': NVARCHAR(3850),
}

prepare_and_insert(productreviews, productreviews_dtypes, 'ProductReview', united_outdoors_engine, { 'PRODUCTREVIEW_PRODUCTREVIEW_ProductID' : products_nk_sk_dict})

Replacing natural keys with surrogate keys for column: PRODUCTREVIEW_PRODUCTREVIEW_ProductID
Inserting data into table: ProductReview with chunk size: 285


## Constraints
altering the tables to add the (foreign key) constraints

In [ ]:
# opening the UnitedOutdoors_constraints.sql file
with open('sql/UnitedOutdoors_constraints.sql', 'r') as file:
    sql_script = file.read()

# Execute the script
split_and_execute_sql_script(sql_script, united_outdoors_engine)

## Closing connections

In [ ]:
try:
    united_outdoors_engine.dispose()
    northwind_engine.dispose()
    aenc_engine.dispose()
    adventureworks_engine.dispose()
except OperationalError as e:
        print(f'Error: {e}')

# Time elapsed

In [ ]:
end_time = time.time()

print(f'Time elapsed: {end_time - start_time} seconds')

Time elapsed: 695.1951155662537 seconds
